# Cobalt Critical Analytics
* Data Sources
    * Google Analytics
        * Metric Categories: pageviews, visitors (not accurate), events/clicks (not captured - navigation, links, content, etc.)
        * Access: API
        * **update 2023/06 - GA4 and other Google products no longer permitted**
            * Existing data must be migrated; future (equivalent) data must be captured server-side or via UPHS/HIPAA approved third parties
    * Cobalt Native Backend
        * Metrics Categories: appointments, assessments, interactions, group sessions, content
        * Access: PSQL Database credentials, white-listed IP address
* Analytics Domains and Metric Categories:
    * Site-Wide: Visitors, Page Views,Navigation,Crisis Calls
    * Connect with Support: Visitors, Page Views, Appointments, Assessments, Interactions
    * In the Studio: Visitors, Page Views, Group Sessions
    * On Your Time: Visitors, Page Views, Content

In [1]:
# Modules
from cobalt_reporting_data_truncated import *

import pandas as pd
import numpy as np
from scipy import stats

import re
import copy
import math
import datetime
from collections import defaultdict

import glob
import os

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns; 
sns.set()

%matplotlib inline

# Pandas view options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
pd.set_option('precision', 4)

147


cobalt_reporting_data_truncated.py:105: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  account['week'] = account['created'].dt.week


# Program Parameters

In [2]:
# Constants
SECONDS_DAY = 60*60*24
YEAR_END = '/12/31'
YEAR_START = '/01/01'

In [3]:
REPORT_PATH = OUTPUT_PATH + 'critical_analytics_output/'

# Extract Data

In [4]:
read_cursor,reporting_cursor = database_connect()

In [5]:
# List all Cobalt tables
query = """SELECT table_name FROM information_schema.tables WHERE table_schema='cobalt'"""
reporting_cursor.execute(query)
result = reporting_cursor.fetchall()

print(len(result))
sorted(result)

147


[('account',),
 ('account_calendar_permission',),
 ('account_fingerprint',),
 ('account_invite',),
 ('account_login_rule',),
 ('account_session',),
 ('account_session_answer',),
 ('account_session_grouping',),
 ('account_session_grouping_all',),
 ('account_source',),
 ('activity_action',),
 ('activity_tracking',),
 ('activity_type',),
 ('answer',),
 ('answer_category',),
 ('appointment',),
 ('appointment_interaction_instance',),
 ('appointment_reason',),
 ('appointment_reason_type',),
 ('appointment_type',),
 ('appointment_type_assessment',),
 ('assessment',),
 ('assessment_type',),
 ('assessment_viewer',),
 ('attendance_status',),
 ('audit_log',),
 ('audit_log_event',),
 ('beta_feature',),
 ('beta_feature_alert',),
 ('beta_status',),
 ('calendar_permission',),
 ('category',),
 ('category_mapping',),
 ('clinic',),
 ('content',),
 ('content_action',),
 ('content_category',),
 ('content_type',),
 ('crisis_contact',),
 ('epic_appointment_filter',),
 ('epic_department',),
 ('external_group

## Accounts

In [6]:
print(len(account))
print(len(account.columns))
account.head(2)

50100
52


,account_id,role_id,institution_id,account_source_id,sso_id,first_name,last_name,display_name,email_address,phone_number,sso_attributes,consent_form_accepted,consent_form_accepted_date,locale,time_zone,created,last_updated,_deprecated_epic_patient_id,_deprecated_epic_patient_id_type,_deprecated_epic_patient_created_by_cobalt,password,source_system_id,provider_id,beta_status_id,access_token_expiration_in_minutes,access_token_short_expiration_in_minutes,metadata,scheduling_tutorial_viewed,gender_identity_id,ethnicity_id,birth_sex_id,race_id,birthdate,consent_form_rejected_date,microsoft_id,epic_patient_mrn,institution_location_id,prompted_for_institution_location,epic_patient_fhir_id,active,test_account,epic_patient_unique_id,epic_patient_unique_id_type,username,password_reset_required,year,month,week,day,year_month,year_month_week,dayofyear
0,afe7ff25-775e-4518-822b-a5bb56683d7d,PATIENT,PENN,ANONYMOUS,None,None,None,None,None,None,None,False,NaT,en-US,America/New_York,2021-04-08 14:08:38.598732+00:00,2021-04-08 14:08:38.598732+00:00,None,None,False,None,COBALT,None,UNKNOWN,NaN,NaN,None,False,NOT_ASKED,NOT_ASKED,NOT_ASKED,NOT_ASKED,None,None,None,None,None,False,None,True,False,None,None,None,False,2021,4,14,8,2021-04-01,2021-04-08,20210408
1,3115d538-f423-4618-b23b-ec6ac9e554a9,PATIENT,PENN,ANONYMOUS,None,None,None,None,None,None,None,False,NaT,en-US,America/New_York,2021-05-18 10:00:00.228881+00:00,2021-05-18 10:00:00.228881+00:00,None,None,False,None,COBALT,None,UNKNOWN,NaN,NaN,None,False,NOT_ASKED,NOT_ASKED,NOT_ASKED,NOT_ASKED,None,None,None,None,None,False,None,True,False,None,None,None,False,2021,5,20,18,2021-05-01,2021-05-13,20210518


## Appointments

In [7]:
print(len(appointment))
print(len(appointment.columns))
appointment.head(2)

5117
56


,appointment_id,provider_id,account_id,acuity_appointment_id,acuity_appointment_type_id,acuity_class_id,bluejeans_meeting_id,videoconference_url,title,start_time,end_time,duration_in_minutes,time_zone,canceled,canceled_at,created,last_updated,epic_contact_id,epic_contact_id_type,appointment_type_id,phone_number,videoconference_platform_id,appointment_reason_id,created_by_account_id,comment,attendance_status_id,bluejeans_participant_passcode,intake_assessment_id,scheduling_system_id,canceled_for_reschedule,rescheduled_appointment_id,intake_account_session_id,interaction_instance_id,patient_reminder_scheduled_message_id,patient_order_id,epic_appointment_fhir_id,appointment_cancelation_reason_id,epic_appointment_fhir_identifier_system,epic_appointment_fhir_identifier_value,epic_appointment_fhir_stu3_response,microsoft_teams_meeting_id,created_year,created_month,created_day,created_year_month,created_year_month_week,created_dayofyear,apt_year,apt_month,apt_day,apt_year_month,apt_year_month_week,apt_dayofyear,created_completed_time,appointment_type_name,support_role_id
0,98365dba-f07b-4e89-b4aa-2eca8d37e428,c236acc5-9473-457a-af51-de57bddcb8a3,90c9b318-a1be-4b67-b5fb-a7a68559e0e3,8.4849e+08,NaN,NaN,1.0494e+08,https://bluejeans.com/698184757/7722,1:1 Appointment with Psychotherapist,2022-07-06 16:00:00-04:00,2022-07-06 17:00:00,60,America/New_York,False,NaT,2022-06-22 16:25:35.721374-04:00,2022-06-22 20:25:35.721374+00:00,None,None,2294b4f1-7fad-454f-a118-06b479356d7d,None,BLUEJEANS,3c34a096-2f4d-4091-a9d8-f0381ba079a1,90c9b318-a1be-4b67-b5fb-a7a68559e0e3,None,UNKNOWN,7722,5929588e-ee53-4df5-9c62-1656a43ff4cd,ACUITY,False,None,b762db93-d3fd-4933-a856-c56126db5a37,None,None,None,None,UNSPECIFIED,None,None,None,None,2022,6,22,2022-06-01,2022-06-16,20220622,2022,7,6,2022-07-01,2022-06-30,20220706,14.0,1:1 Appointment with Psychotherapist,CLINICIAN
0,8e52c173-f3d4-4b9d-913c-32a5679e2387,5fc46a43-58d1-46fc-bae9-44c65e58388b,9a64f46d-ca02-4783-9f75-d9f708a79fa9,8.0982e+08,NaN,NaN,1.0305e+08,https://bluejeans.com/166022552/0525,1:1 with Care Manager,2022-04-22 11:00:00-04:00,2022-04-22 11:30:00,30,America/New_York,False,NaT,2022-04-20 11:32:48.204541-04:00,2022-04-20 15:32:48.204541+00:00,None,None,aef3538a-73fa-40cf-8d3f-367022913b9c,None,BLUEJEANS,3c34a096-2f4d-4091-a9d8-f0381ba079a1,9a64f46d-ca02-4783-9f75-d9f708a79fa9,None,UNKNOWN,0525,20290377-2978-4472-b00c-6f604f25ee58,ACUITY,False,None,08048f29-035c-4341-82fd-b41406303114,None,None,None,None,UNSPECIFIED,None,None,None,None,2022,4,20,2022-04-01,2022-04-14,20220420,2022,4,22,2022-04-01,2022-04-21,20220422,2.0,1:1 with Care Manager,CARE_MANAGER


## Group Sessions

In [8]:
print(len(group_session))
print(len(group_session.columns))
group_session.head(2)

655
45


,group_session_id,institution_id,group_session_status_id,assessment_id,title,description,facilitator_account_id,facilitator_name,facilitator_email_address,videoconference_url,start_date_time,end_date_time,seats,url_name,confirmation_email_content,locale,time_zone,created,last_updated,group_session_scheduling_system_id,send_followup_email,followup_email_content,followup_email_survey_url,submitter_account_id,en_search_vector,target_email_address,screening_flow_id,visible_flag,group_session_collection_id,followup_time_of_day,followup_day_offset,send_reminder_email,reminder_email_content,single_session_flag,date_time_description,group_session_learn_more_method_id,learn_more_description,different_email_address_for_notifications,group_session_location_type_id,in_person_location,override_platform_name,override_platform_email_image_url,override_platform_support_email_address,registration_end_date_time,image_file_upload_id
0,f5b403f4-037d-44d8-b6ac-4bab56ce21f4,PENN,ARCHIVED,None,Relationships That Hurt: Navigating Intimate P...,<p>We will discuss common experiences of intim...,None,Dr. Lily Brown,lilybr@pennmedicine.upenn.edu,https://teams.microsoft.com/l/meetup-join/19%3...,2024-02-06 12:00:00,2024-02-06 12:30:00,200.0,relationships-that-hurt-navigating-intimate-pa...,<p><br></p>,en-US,America/New_York,2024-01-16 17:18:23.313779+00:00,2024-02-06 17:30:22.710636+00:00,COBALT,True,<p>Good morning! Thank you for recently attend...,https://redcap.med.upenn.edu/surveys/?s=F3RRD8...,d18daeab-5f7f-4be6-b2fc-8e47b0a09402,'brown':40 'call':18 'common':11 'discuss':10 ...,janna.rothschild@pennmedicine.upenn.edu,aba782b0-c89b-47c6-b74b-2d75685916aa,True,None,08:00:00,1.0,True,<p><br></p>,True,None,None,None,True,VIRTUAL,None,None,None,None,NaT,e7b53dac-6d9e-456a-a3a8-0f5a0aeb6ed3
1,98901804-b5b1-4363-8e85-9fecfcd4503c,PENN,ARCHIVED,None,Lunchtime Yoga- All levels,<p>This 60-minute lunchtime yoga is open to Pe...,d18daeab-5f7f-4be6-b2fc-8e47b0a09402,Judy Glass,judith.glass@pennmedicine.upenn.edu,https://us02web.zoom.us/meeting/register/tZAkc...,2023-09-01 12:00:00,2023-09-01 13:00:00,75.0,lunchtime-yoga-4-2023-09-01,None,en-US,America/New_York,2023-07-31 14:54:03.295574+00:00,2024-02-02 15:25:25.919577+00:00,COBALT,False,None,None,d18daeab-5f7f-4be6-b2fc-8e47b0a09402,'-01':71 '-09':70 '-2023':69 '4':68 '60':6 'da...,judith.glass@pennmedicine.upenn.edu,aba782b0-c89b-47c6-b74b-2d75685916aa,True,None,None,NaN,False,None,True,None,None,None,False,VIRTUAL,None,None,None,None,NaT,3ba62d98-1211-4bd3-b0c7-bb5c3196363e


In [9]:
print(len(group_session_request))
print(len(group_session_request.columns))
group_session_request.head(2)

10
17


,group_session_request_id,institution_id,group_session_request_status_id,title,description,facilitator_account_id,facilitator_name,facilitator_email_address,image_url,url_name,custom_question_1,custom_question_2,created,last_updated,submitter_account_id,en_search_vector,data_collection_enabled
0,c6c79e7b-ab4d-4def-8acf-d4f320878c70,PENN,DELETED,Wellfocused's Maintaining Physical Wellness,While fitness and nutrition have always been e...,None,Kelley Kugler,rachel.kishton@pennmedicine.upenn.edu,https://penncobalt.s3.us-east-2.amazonaws.com/...,physical-wellness,What would you like the facilitator to focus o...,None,2020-12-10 15:21:37.555285+00:00,2023-05-10 12:24:00.274334+00:00,b51a9e56-2a34-4a93-b32c-bab3030ff32b,'-19':21 'adequ':47 'adjust':80 'alway':11 'ch...,True
1,189281a0-ba53-4b7d-9d74-3fe32330c6b1,PENN,ADDED,Wellfocused's Maintaining Physical Wellness - ...,While fitness and nutrition have always been e...,None,Mitchell Sherman,mitchell.sherman@pennmedicine.upenn.edu,https://penncobalt.s3.us-east-2.amazonaws.com/...,physical-wellness,"What, in particular, would you like to discuss...",None,2021-01-12 19:47:30.214032+00:00,2022-09-20 15:10:28.272552+00:00,b51a9e56-2a34-4a93-b32c-bab3030ff32b,'-19':24 'adequ':47 'adjust':80 'alway':14 'ch...,True


In [10]:
print(len(group_session_reservation))
print(len(group_session_reservation.columns))
group_session_reservation.head(2)

15213
10


,group_session_reservation_id,group_session_id,account_id,canceled,created,last_updated,followup_email_status_id,followup_email_sent_timestamp,attendee_reminder_scheduled_message_id,attendee_followup_scheduled_message_id
0,cadb0450-b405-428f-ac04-7b897142781e,991069a0-374f-4142-8e13-d4af9885d176,4b64f47a-f321-4b0e-839e-d2e721975c78,False,2021-01-15 00:01:46.592444+00:00,2021-01-15 00:01:46.592444+00:00,NOT_SENT,NaT,None,None
1,81e71aa2-72a4-47d9-b16d-5a55bc47a29f,dc67a1bd-e3db-44ac-b55e-83bd9c44e500,b51a9e56-2a34-4a93-b32c-bab3030ff32b,True,2020-12-10 15:44:00.874111+00:00,2020-12-10 15:55:54.917334+00:00,NOT_SENT,NaT,None,None


## Content and Activity Tracking

In [11]:
print(len(content))
print(len(content.columns))
content.head(2)

695
22


,content_id,content_type_id,title,url,date_created,description,author,created,last_updated,owner_institution_id,deleted_flag,duration_in_minutes,en_search_vector,never_embed,shared_flag,search_terms,publish_start_date,publish_end_date,publish_recurring,published,file_upload_id,image_file_upload_id
0,c96b5665-6235-443b-8a29-54575cda1166,ARTICLE,The Psychology of Relationships: Connections f...,https://www.psych.on.ca/Public/Blog/2023/the-p...,2024-02-21 16:30:33.480895+00:00,"<p>What is a relationship, and what are the ps...",Ontario Psychological Association,2024-02-21 16:30:33.480895+00:00,2024-02-28 20:33:10.114538+00:00,PENN,False,5.0,'activ':70 'addict':191 'anoth':54 'anxieti':1...,False,True,None,2024-02-21 00:00:00+00:00,None,False,True,None,b5ab4bbc-5d21-4a3a-a11d-8165feecb56a
1,beacf034-fc96-4ff2-880a-0c00c802f04d,ARTICLE,Winding Down Before Bed During COVID,None,2021-03-09 19:12:44.706480+00:00,<p><strong>Winding down before bedtime during ...,"Dr. Jennifer Goldschmied, PhD",2021-03-09 19:12:44.706480+00:00,2024-02-02 15:25:25.919577+00:00,PENN,False,5.0,"'-19':13,34 '-60':50 '30':49,166 '65':441 'abr...",False,False,None,2021-03-09 19:12:44.706480+00:00,None,False,False,None,efa389a5-827f-4f90-b006-a430d3d7090a


In [12]:
print(len(activity_tracking))
print(len(activity_tracking.columns))
activity_tracking.head(2)

39888
9


,activity_tracking_id,account_id,activity_type_id,activity_action_id,created,last_updated,session_tracking_id,context,activity_key
0,1d373d36-07a5-4646-a97b-ff5010e29d0a,a924fd84-57c5-465a-8343-b4708535ba48,ACCOUNT,CREATE,2021-12-14 23:51:39.516663+00:00,2021-12-14 23:51:39.516663+00:00,None,{'accountId': 'a924fd84-57c5-465a-8343-b470853...,None
1,57732d9a-f8df-4baf-a85e-86a6c66626fb,312aeafc-dcaf-4a1b-bf92-b3fe64328f14,ACCOUNT,CREATE,2021-12-14 23:52:08.878310+00:00,2021-12-14 23:52:08.878310+00:00,None,{'accountId': '312aeafc-dcaf-4a1b-bf92-b3fe643...,None


In [13]:
# Get content activity
content_activity = activity_tracking[activity_tracking['activity_type_id']=='CONTENT'].copy()
content_activity.loc[:,'context'] = content_activity['context'].apply(lambda x: x['contentId']).copy()
content_activity = content_activity.rename(columns={'context':'content_id', 
                                                    'created':'view_created', 
                                                    'last_updated':'view_last_updated'})
content_activity = content_activity.merge(content, how='inner', left_on='content_id', right_on='content_id')

In [14]:
print(len(content_activity))
print(len(content_activity.columns))
content_activity.head(2)

22291
30


,activity_tracking_id,account_id,activity_type_id,activity_action_id,view_created,view_last_updated,session_tracking_id,content_id,activity_key,content_type_id,title,url,date_created,description,author,created,last_updated,owner_institution_id,deleted_flag,duration_in_minutes,en_search_vector,never_embed,shared_flag,search_terms,publish_start_date,publish_end_date,publish_recurring,published,file_upload_id,image_file_upload_id
0,6bf67b64-e5ec-4840-9eb5-77c2df7c38ce,d2ea1037-ebf0-4980-a319-457ae4a59f5f,CONTENT,VIEW,2021-12-20 13:15:06.132125+00:00,2021-12-20 13:15:06.132125+00:00,ca553422-12ba-4f7e-9f79-695d37b47261,8c096651-a71e-4544-bf85-66bd4c254f51,None,PODCAST,"The Wellfocused Podcast - Sn. 2, Ep. 4: Living...",https://www.youtube.com/watch?v=F6VuZSyTr8A&li...,2021-11-05 13:03:41.646886+00:00,"Jen Brady, Associate Director of Benefits and ...",Mitchell Sherman,2021-11-05 13:03:41.646886+00:00,2024-02-02 15:25:25.919577+00:00,PENN,False,22.0,'-19':41 '2':5 '4':7 'adopt':49 'also':25 'ass...,False,False,None,2021-11-05 13:03:41.646886+00:00,None,False,False,None,62d2ad84-b487-4c16-8e2c-f8f98a8aa820
1,60ab079a-0c95-4355-97dc-ce2f57e235d2,b45acabf-c5be-4b3d-b503-7eb19bd86bae,CONTENT,VIEW,2022-01-19 18:49:13.488345+00:00,2022-01-19 18:49:13.488345+00:00,40fda0e7-c371-42bb-9e69-d9ad91f1f595,8c096651-a71e-4544-bf85-66bd4c254f51,None,PODCAST,"The Wellfocused Podcast - Sn. 2, Ep. 4: Living...",https://www.youtube.com/watch?v=F6VuZSyTr8A&li...,2021-11-05 13:03:41.646886+00:00,"Jen Brady, Associate Director of Benefits and ...",Mitchell Sherman,2021-11-05 13:03:41.646886+00:00,2024-02-02 15:25:25.919577+00:00,PENN,False,22.0,'-19':41 '2':5 '4':7 'adopt':49 'also':25 'ass...,False,False,None,2021-11-05 13:03:41.646886+00:00,None,False,False,None,62d2ad84-b487-4c16-8e2c-f8f98a8aa820


In [15]:
# Get content view counts
popular_content = content_activity.groupby(['content_id'])[['activity_tracking_id']].count()
popular_content = popular_content.rename(columns={'activity_tracking_id':'view_count'})
popular_content = content.merge(popular_content, how='inner', left_on='content_id', right_index=True)

# popular content
popular_content = popular_content.sort_values(['view_count'], ascending=False).reset_index(drop=True)

# popular content grouped by type
popular_content_grouped = popular_content.sort_values(['content_type_id','view_count'], ascending=False).reset_index(drop=True)


In [16]:
print(len(popular_content))
print(len(popular_content.columns))
popular_content.head(2)

505
23


,content_id,content_type_id,title,url,date_created,description,author,created,last_updated,owner_institution_id,deleted_flag,duration_in_minutes,en_search_vector,never_embed,shared_flag,search_terms,publish_start_date,publish_end_date,publish_recurring,published,file_upload_id,image_file_upload_id,view_count
0,09ab5264-3d8f-4e8e-be15-074e38528bf1,VIDEO,Tackling Burnout,https://youtu.be/okweTcWwe8c,2021-06-15 00:00:00+00:00,"Dr. Laura Hamill, Chief Science Officer at Lim...",Dr. Laura Hamill,2021-06-15 20:08:10.643692+00:00,2024-02-02 15:25:25.919577+00:00,PENN,False,60.0,"'also':33 'behind':15 'burnout':2,16,63 'caus'...",False,False,None,2021-06-15 00:00:00+00:00,None,False,True,None,90428f87-6406-4d14-b06f-a6a15c61c61c,792
1,0e722b38-b101-4a8c-8a2e-f34ff19137ea,PODCAST,Managing Compassion Fatigue,https://youtu.be/8JcVqJjZTbY,2022-01-24 15:05:57.739142+00:00,"We have been dipping extensively into our ""wel...","Lily Brown, PhD",2022-01-24 15:05:57.739142+00:00,2024-02-02 15:25:25.919577+00:00,PENN,False,22.0,"'20':57 'anxieti':72 'ask':30 'brown':62,95,11...",False,False,None,2022-01-24 15:05:57.739142+00:00,None,False,True,None,5ec98d65-dbcb-4579-ae1d-55a76dfd9159,770


In [17]:
print(len(popular_content_grouped))
print(len(popular_content_grouped.columns))
popular_content_grouped.head(2)

505
23


,content_id,content_type_id,title,url,date_created,description,author,created,last_updated,owner_institution_id,deleted_flag,duration_in_minutes,en_search_vector,never_embed,shared_flag,search_terms,publish_start_date,publish_end_date,publish_recurring,published,file_upload_id,image_file_upload_id,view_count
0,97c68115-345b-45d1-bd4d-5663df9c155d,WORKSHEET,Return Home Safely,https://penncobalt.s3.us-east-2.amazonaws.com/...,2020-04-09 00:00:00+00:00,Crowdsourced tips and tricks for frontline sta...,Center for Health Care Innovation,2020-04-13 16:12:33.624471+00:00,2024-02-02 15:25:25.919577+00:00,PENN,False,8.0,'care':37 'center':34 'clinician':19 'crowdsou...,False,False,None,2020-04-09 00:00:00+00:00,None,False,False,None,18e93a7a-a1a5-4b9f-aada-ba9b5e45727c,184
1,c93a54bc-6c1f-452b-a1a2-c4bb63c310ac,WORKSHEET,Mourner's Bill of Rights,https://www.centerforloss.com/wp-content/uploa...,2020-04-30 14:15:45.805947+00:00,COVID Mourner's Bill of Rights,Dr. Alan Wolfelt,2020-04-30 14:15:45.805947+00:00,2024-02-02 15:25:25.919577+00:00,PENN,False,5.0,"'alan':13 'bill':3,9 'covid':6 'dr':12 'mourne...",False,False,None,2020-04-30 14:15:45.805947+00:00,None,False,True,None,None,68


In [18]:
# content summary
popular_content_summary = pd.concat([popular_content.groupby(['content_type_id'])['view_count'].count(),
                                     popular_content.groupby(['content_type_id'])['view_count'].sum()], axis=1)
popular_content_summary.columns = ['content_count', 'view_count']
popular_content_summary['views_per_content'] = popular_content_summary['view_count']/popular_content_summary['content_count']

In [19]:
print(len(popular_content_summary))
print(len(popular_content_summary.columns))
popular_content_summary.head(10)

5
3


,content_count,view_count,views_per_content
content_type_id,,,
APP,5,73,14.6000
ARTICLE,271,5672,20.9299
PODCAST,132,9748,73.8485
VIDEO,93,6499,69.8817
WORKSHEET,4,299,74.7500


# Crosstab
* Accounts by source 
* Recommendations by escalation level 
* Appointments by provider type and/or clinical level

## Prepare Crosstab Data

### Parameters

In [20]:
# Days in each month
month_ids = list(range(1,13))
month_days = [31,28,31,30,31,30,31,31,30,31,30,31]
month_days_dict = {idx:days for idx,days in zip(month_ids,month_days)}
avg_days_month = np.mean(list(month_days_dict.values()))

# Cobalt date range
delta_months_dict = {}
date_keys = [2020,2021,2022,'All Time']
start_dates = [cobalt_start_date, 
               pd.to_datetime('2021/01/01').tz_localize(tz=COBALT_TZ),
               pd.to_datetime('2022/01/01').tz_localize(tz=COBALT_TZ),
               cobalt_start_date]
end_dates = [pd.to_datetime('2020/12/31').tz_localize(tz=COBALT_TZ),
             pd.to_datetime('2021/12/31').tz_localize(tz=COBALT_TZ),
             current_date,
             current_date]

for key,start,end in zip(date_keys,start_dates,end_dates):
    delta = (end-start).days+(end-start).seconds/SECONDS_DAY
    delta_months = delta/avg_days_month
    delta_months_dict[key] = delta_months

### Report Data

In [21]:
# Get screening tables
screening = get_table_data(reporting_cursor, 'screening')
screening_session = get_table_data(reporting_cursor, 'screening_session')
screening_session_screening = get_table_data(reporting_cursor, 'screening_session_screening')
screening_session_support_role_recommendation = get_table_data(reporting_cursor, 'screening_session_support_role_recommendation')

# Create dictionary for PHQ-9, GAD-7, and WHO-5
screening_id_dict = dict(screening[screening['name'].isin(['WHO-5','PHQ-9','GAD-7'])][['active_screening_version_id','name']].values)
screening_id_dict = dict(([[item[0],item[1]+suffix] for item,suffix in zip(sorted(screening_id_dict.items(),key=lambda x: x[1]),['_1','_2','_1','_2','_1'])]))
screening_name_dict = {v:k for k,v in screening_id_dict.items()}
# Reformat screening session screening and scores so they are useable
screening_scores = screening_session_screening[screening_session_screening['screening_version_id'].isin(screening_id_dict.keys())].copy()
screening_scores['screening_name'] = screening_scores['screening_version_id'].map(screening_id_dict)
screening_scores['screening_score'] = screening_scores['score'].apply(lambda x: x['overallScore'])

# Merge screening_session and screening_session_screening data to make useable dataset
screening_data_master = screening_scores.merge(screening_session,how='inner',on='screening_session_id',
                                               suffixes=('_screening','_session'),indicator='source')
screening_data_master = screening_data_master.merge(screening_session_support_role_recommendation, how='inner', on='screening_session_id')

In [22]:
# Display updated table
print(len(screening_data_master))
print(len(screening_data_master.columns))
screening_data_master.head(2)

1834
34


,screening_session_screening_id,screening_session_id,screening_version_id,screening_order,valid,completed_screening,legacy_score,created_screening,last_updated_screening,score,below_scoring_threshold,screening_name,screening_score,screening_flow_version_id,target_account_id,created_by_account_id,completed_session,crisis_indicated,created_session,last_updated_session,skipped,skipped_at,completed_at,crisis_indicated_at,patient_order_id,group_session_id,account_check_in_action_id,metadata,source,screening_session_support_role_recommendation_id,support_role_id,weight,created,last_updated
0,4a0c1b35-1039-4716-b831-fdc6d5ac2f41,b550be27-294a-4473-83c4-64cca425c1cc,2cc00d3a-ac38-4ec4-8e2d-67c829e7471c,3,True,False,NaN,2022-10-21 17:48:42.758882+00:00,2022-10-21 17:48:42.758882+00:00,{'overallScore': 0},None,GAD-7_1,0,47449be0-3496-45e6-8121-8a82347b307d,72442213-35b5-41c9-b839-a189d0335f5e,72442213-35b5-41c9-b839-a189d0335f5e,True,True,2022-10-21 17:48:15.571967+00:00,2022-10-21 17:48:40.459493+00:00,False,NaT,2022-10-21 17:48:40.459493+00:00,2022-10-21 17:48:37.885159+00:00,None,None,None,None,both,cc9f08de-784b-4784-9f23-02683055d116,CARE_MANAGER,1,2022-10-21 17:48:40.459493+00:00,2022-10-21 17:48:40.459493+00:00
1,dec45628-7411-4f5c-b2a9-d1fb3b500d81,b550be27-294a-4473-83c4-64cca425c1cc,5a25b1e6-b6c6-49ad-8daa-c33d9140b74a,1,True,True,NaN,2022-10-21 17:48:15.571967+00:00,2022-10-21 17:48:23.007417+00:00,{'overallScore': 0},None,WHO-5_1,0,47449be0-3496-45e6-8121-8a82347b307d,72442213-35b5-41c9-b839-a189d0335f5e,72442213-35b5-41c9-b839-a189d0335f5e,True,True,2022-10-21 17:48:15.571967+00:00,2022-10-21 17:48:40.459493+00:00,False,NaT,2022-10-21 17:48:40.459493+00:00,2022-10-21 17:48:37.885159+00:00,None,None,None,None,both,cc9f08de-784b-4784-9f23-02683055d116,CARE_MANAGER,1,2022-10-21 17:48:40.459493+00:00,2022-10-21 17:48:40.459493+00:00


If someone does not complete all required assessments according to the established escalation logic, the severity is inconclusive. 
If someone scores higher than 12 on WHO-5, the severity is mild. They do not complete any further assessments.  
If someone scores higher than 0 on PHQ-9 Q9, a crisis flag is triggered. They do not complete any further assessments.  
If someone scores less than 10 on PHQ-9 and less than 10 on GAD-7, the severity is mild. 
If someone scores higher than 19 on PHQ-9 or higher than 19 on GAD-7, the severity is severe. 
If someone’s scores are not covered by above rules, the severity is moderate.  


In [55]:
screening_scores_dict = dict()
for name in screening_data_master.screening_name.unique():
    screening_scores_dict[name] = screening_data_master[screening_data_master['screening_name']==name]['screening_score']

In [64]:
pd.concat([screening_scores_dict['GAD-7_1'],
           screening_scores_dict['WHO-5_1'],
           screening_scores_dict['PHQ-9_1']], axis=0).sort_index()

0        0
1        0
2       24
3       13
4        9
        ..
1785     8
1786     7
1787     7
1788    11
1789     5
Name: screening_score, Length: 1790, dtype: int64

In [75]:
screening_data_master.completed_session.value_counts()

True    1790
Name: completed_session, dtype: int64

In [79]:
screening_data_master.completed_screening.value_counts()

True     1670
False     120
Name: completed_screening, dtype: int64

In [80]:
screening_data_master.crisis_indicated.value_counts()

False    1527
True      263
Name: crisis_indicated, dtype: int64

In [93]:
screening_data_master[screening_data_master['crisis_indicated']==True].completed_screening.value_counts()

True     165
False     98
Name: completed_screening, dtype: int64

In [92]:
screening_data_master[screening_data_master['completed_screening']==False]

,screening_session_screening_id,screening_session_id,screening_version_id,screening_order,valid,completed_screening,legacy_score,created_screening,last_updated_screening,score,below_scoring_threshold,screening_name,screening_score,screening_flow_version_id,target_account_id,created_by_account_id,completed_session,crisis_indicated,created_session,last_updated_session,skipped,skipped_at,completed_at,crisis_indicated_at,patient_order_id,group_session_id,account_check_in_action_id,source,screening_session_support_role_recommendation_id,support_role_id,weight,created,last_updated
0,4a0c1b35-1039-4716-b831-fdc6d5ac2f41,b550be27-294a-4473-83c4-64cca425c1cc,2cc00d3a-ac38-4ec4-8e2d-67c829e7471c,3,True,False,NaN,2022-10-21 17:48:42.758882+00:00,2022-10-21 17:48:42.758882+00:00,{'overallScore': 0},None,GAD-7_1,0,47449be0-3496-45e6-8121-8a82347b307d,72442213-35b5-41c9-b839-a189d0335f5e,72442213-35b5-41c9-b839-a189d0335f5e,True,True,2022-10-21 17:48:15.571967+00:00,2022-10-21 17:48:40.459493+00:00,False,NaT,2022-10-21 17:48:40.459493+00:00,2022-10-21 17:48:37.885159+00:00,None,None,None,both,cc9f08de-784b-4784-9f23-02683055d116,CARE_MANAGER,1,2022-10-21 17:48:40.459493+00:00,2022-10-21 17:48:40.459493+00:00
12,470a128b-b3ea-4676-9b34-91243a17a5f6,fc6a2403-23de-4e74-b0fe-1dc02fda360c,2cc00d3a-ac38-4ec4-8e2d-67c829e7471c,3,True,False,NaN,2022-10-20 20:51:54.866193+00:00,2022-10-20 20:51:54.866193+00:00,{'overallScore': 0},None,GAD-7_1,0,47449be0-3496-45e6-8121-8a82347b307d,839b09d7-0cb4-4200-86f1-687055480b56,839b09d7-0cb4-4200-86f1-687055480b56,True,True,2022-10-20 19:21:01.797424+00:00,2022-10-20 20:50:41.197979+00:00,False,NaT,2022-10-20 20:50:41.197979+00:00,2022-10-20 20:50:41.197979+00:00,None,None,None,both,0da006d3-3494-4996-aa03-b3002def1b75,CARE_MANAGER,1,2022-10-20 20:50:41.197979+00:00,2022-10-20 20:50:41.197979+00:00
53,5d8337de-6b8c-4429-b9f3-5ec85258cbb4,fb25f0d5-dd9f-4341-a9c3-b1dbb681ffba,2cc00d3a-ac38-4ec4-8e2d-67c829e7471c,3,True,False,NaN,2022-11-26 21:51:24.583921+00:00,2022-11-26 21:51:24.583921+00:00,{'overallScore': 0},None,GAD-7_1,0,47449be0-3496-45e6-8121-8a82347b307d,b8a742d3-194f-46de-8f65-5122a4bf1a0f,b8a742d3-194f-46de-8f65-5122a4bf1a0f,True,True,2022-11-26 21:50:31.469328+00:00,2022-11-26 21:51:32.821578+00:00,True,2022-11-26 21:51:32.821578+00:00,2022-11-26 21:51:32.821578+00:00,2022-11-26 21:51:10.628206+00:00,None,None,None,both,594a3c43-70cc-4d84-a853-2c529e3c459e,CARE_MANAGER,1,2022-11-26 21:51:10.628206+00:00,2022-11-26 21:51:10.628206+00:00
63,69ea9c9e-61ed-4d48-9bd2-e18b4f1e7875,b74562bf-7c92-48ae-8753-bf29bd1c99d0,a4732277-0d3c-4651-8181-27b06aed7c72,2,False,False,NaN,2022-12-12 15:25:00.182555+00:00,2022-12-12 15:25:07.551449+00:00,{'overallScore': 0},None,PHQ-9_1,0,47449be0-3496-45e6-8121-8a82347b307d,7bc144e9-2968-425f-9080-fec84af4b35e,7bc144e9-2968-425f-9080-fec84af4b35e,True,False,2022-12-12 15:24:26.081114+00:00,2022-12-12 15:26:36.589450+00:00,False,NaT,2022-12-12 15:26:36.589450+00:00,NaT,None,None,None,both,c2ffdf7c-96fe-42c0-9882-23325a00258c,COACH,1,2022-12-12 15:26:36.589450+00:00,2022-12-12 15:26:36.589450+00:00
79,6a716ae8-5235-4108-bddb-ebbbf780a762,e16983c0-7302-4b02-8001-47948992a3c0,a4732277-0d3c-4651-8181-27b06aed7c72,2,True,False,NaN,2022-12-27 03:02:15.432194+00:00,2022-12-27 03:04:27.057473+00:00,{'overallScore': 18},None,PHQ-9_1,18,47449be0-3496-45e6-8121-8a82347b307d,e1c1bba3-054d-483a-a78b-899446e21a10,e1c1bba3-054d-483a-a78b-899446e21a10,True,True,2022-12-27 03:01:15.709274+00:00,2022-12-27 03:03:51.164478+00:00,False,NaT,2022-12-27 03:03:51.164478+00:00,2022-12-27 03:03:51.164478+00:00,None,None,None,both,ef7d07c5-c0a8-4206-98f0-5b7cafe142e5,CARE_MANAGER,1,2022-12-27 03:03:51.164478+00:00,2022-12-27 03:03:51.164478+00:00
80,8c2c7a64-c59e-4cb4-a4a8-d3999587732d,e16983c0-7302-4b02-8001-47948992a3c0,2cc00d3a-ac38-4ec4-8e2d-67c829e7471c,3,False,False,NaN,2022-12-27 03:04:13.408298+00:00,2022-12-27 03:04:27.057473+00:00,{'overallScore': 0},None,GAD-7_1,0,47449be0-3496-45e6-8121-8

In [67]:
screen_sess.escalation.value_counts()

moderate    1154
mild        1120
severe       220
Name: escalation, dtype: int64

In [68]:
screen_sess.crisis.value_counts()

no     2193
yes     301
Name: crisis, dtype: int64

In [23]:
# Accounts
acct = account.copy()
acct['account_type'] = acct['account_source_id'].apply(lambda x: x if x=='ANONYMOUS' else 'VERIFIED')

# Screening TODO: CONCAT EQUIV DATA FROM NEW SCREENING TABLES
screen_sess = account_instance[account_instance['instance_complete']==1].copy()
screen_sess = screen_sess.groupby(['account_id']).tail(1)
screen_sess['crisis'] = screen_sess['crisis'].apply(lambda x: 'yes' if x==1 else 'no')
screen_sess['year'] = screen_sess['complete_time'].dt.year
screen_sess['month'] = screen_sess['complete_time'].dt.month

# Appointments
appt = appointment.copy()
appt['provider_type'] = appt['support_role_id'].apply(lambda x: 'clinical' if x in ['PSYCHIATRIST','CLINICIAN'] else 'sub-clinical')
appt['year'] = appt['apt_year']
appt['month'] = appt['apt_month']

# Group reservations
group_res = group_session_reservation.copy()
group_res['year'] = group_res['created'].dt.year
group_res['month'] = group_res['created'].dt.month

# Content views
content_views = content_activity.copy()
content_views['year'] = content_views['view_created'].dt.year
content_views['month'] = content_views['view_created'].dt.month

In [24]:
# Display updated table
print(len(screen_sess))
print(len(screen_sess.columns))
screen_sess.head(2)

2495
30


,instance_id,account_id,num_instance_sessions,PHQ4_attempts,PHQ9_attempts,GAD7_attempts,PCPTSD_attempts,PHQ4_complete,PHQ9_complete,GAD7_complete,PCPTSD_complete,PHQ4_score,PHQ9_score,GAD7_score,PCPTSD_score,PHQ4_severity,PHQ9_severity,GAD7_severity,PCPTSD_severity,crisis,crisis_text,crisis_value,instance_complete,escalation,escalation_provider,start_time,complete_time,last_updated,year,month
0,000df8c8-7a8b-4060-a06a-27f65d2d2fd9_0,000df8c8-7a8b-4060-a06a-27f65d2d2fd9,5,1,2,1,1,1,1,1,1,10,12,16,2,severe,moderate,severe,no_symptoms,no,NaN,0,1,moderate,psychotherapist,2022-02-20 01:52:12.942990+00:00,2022-02-19 20:58:59.671561-05:00,2022-02-20 01:58:59.671561+00:00,2022,2
2,00242b81-b10b-4425-84cf-21aa4da6e18d_0,00242b81-b10b-4425-84cf-21aa4da6e18d,4,1,1,1,1,1,1,1,1,3,4,5,2,mild,no_symptoms,mild,no_symptoms,no,NaN,0,1,mild,resilience_coach,2021-01-04 16:35:12.603321+00:00,2021-01-04 11:41:38.151380-05:00,2021-01-04 16:41:38.151380+00:00,2021,1


### Data Structures

In [25]:
# Data entities
data_entities = ['acct','screen_sess','appt','group_res','content_views']
data_entity_names = ['Accounts','Screening Sessions','Appointments','Group Sessions','Content Views']
data_entity_data = [acct,screen_sess,appt,group_res,content_views]
data_entity_acct_ids = [data['account_id'].unique() for data in data_entity_data]
data_entity_keys = ['account_id','instance_id','appointment_id','group_session_reservation_id','activity_tracking_id']
data_entity_col_groups = [{'acct':['account_type','account_source_id']},
                          {'screen_sess':['escalation','crisis']}, 
                          {'appt':['provider_type','support_role_id']}, 
                          {'group_res':[]}, 
                          {'content_views':['content_type_id']}]
data_entity_dict = {entity:{'name':name,'data':data,'acct_ids':acct_ids,'key':key,'col_groups':col_groups[entity]} 
                    for entity,name,data,acct_ids,key,col_groups in 
                    zip(data_entities,data_entity_names,data_entity_data,
                        data_entity_acct_ids,data_entity_keys,data_entity_col_groups)}

# Indicator columns
ind_entities = ['screen_sess','appt','group_res','content_views']
ind_acct_ids = [data_entity_dict[entity]['acct_ids'] for entity in ind_entities]
ind_group_names = ['has_screen_sess', 'has_appt', 'has_group_res', 'has_content_views']
ind_group_labels = [name.replace('_',' ').title() for name in ind_group_names]
ind_entity_dict = {entity:{'acct_ids':acct_ids,'group_name':group_name,'group_label':group_label} 
                   for entity,acct_ids,group_name,group_label in 
                   zip(ind_entities,ind_acct_ids,ind_group_names,ind_group_labels)}

# Index grouping
time_group_names = ['year','month']
time_group_labels = ['Year','Month']
time_group_dict = dict(zip(time_group_names,time_group_labels))

index_group_names = time_group_names + ind_group_names
index_group_labels = time_group_labels + ind_group_labels
index_group_dict = dict(zip(index_group_names,index_group_labels))

# Column grouping
col_group_names = ['Aggregate','account_type','account_source_id',
                   'escalation','crisis',
                   'provider_type','support_role_id',
                   'content_type_id']
col_group_labels = ['Aggregate','Account Type','Account Source',
                    'Triage Level','Crisis Flag',
                    'Clinical Level','Provider Role',
                    'Content Type']
col_group_dict = dict(zip(col_group_names,col_group_labels))
col_level_labels = ['Entity','Group','Metric']

## Generate Master Crosstab

In [26]:
# Create crosstab
crosstab_results = []
crosstab_results_dict = {}

for entity in data_entities:
    name = data_entity_dict[entity]['name']
    data = data_entity_dict[entity]['data']
    acct_ids = data_entity_dict[entity]['acct_ids']
    key = data_entity_dict[entity]['key']
    col_groups = data_entity_dict[entity]['col_groups']
    
    # Construct indicator cols 
    for ind in ind_entities:
        ind_acct_ids = ind_entity_dict[ind]['acct_ids']
        ind_group_name = ind_entity_dict[ind]['group_name']
        ind_group_label = ind_entity_dict[ind]['group_label']
        
        data[ind_group_name] = 0
        data.loc[data['account_id'].isin(ind_acct_ids),[ind_group_name]]=1
    
    # Create base crosstab
    temp_results = []
    base_crosstab = data.groupby(index_group_names)[[key]].count()
    base_crosstab.columns = pd.MultiIndex.from_product([base_crosstab.columns, ['Aggregate'],['Total']])
    temp_results.append(base_crosstab)
    # Add grouping cols
    for col in col_groups: # can add if/else here for str vs list args
        grouping = index_group_names + [col]
        col_crosstab = data.groupby(grouping)[[key]].count().unstack()
        col_crosstab.columns = pd.MultiIndex.from_tuples([(item[0],col,item[1]) for item in col_crosstab.columns])
        temp_results.append(col_crosstab)
    curr_crosstab = pd.concat(temp_results, axis=1).fillna(0)
    
    # Adjust format and labeling
    level_0_names = {key:name}
    level_1_names = {item:col_group_dict[item] for item in curr_crosstab.columns.get_level_values(1)}
    level_2_names = {item:item.lower().capitalize() for item in curr_crosstab.columns.get_level_values(2)}
    curr_crosstab = curr_crosstab.rename(columns=level_0_names, level=0)
    curr_crosstab = curr_crosstab.rename(columns=level_1_names, level=1)
    curr_crosstab = curr_crosstab.rename(columns=level_2_names, level=2)
    curr_crosstab.columns = curr_crosstab.columns.rename(col_level_labels)
    
    # Add data for all time
    levels = curr_crosstab.index.names[1:]
    agg_crosstab = pd.concat({'All Time': curr_crosstab.groupby(axis=0,level=levels).sum()}, names=['year'])
    curr_crosstab = pd.concat([agg_crosstab,curr_crosstab], axis=0)
    
    # Add to result set
    crosstab_results.append(curr_crosstab)
    crosstab_results_dict[entity]=curr_crosstab
    

In [27]:
# Master crosstab
crosstab_master = pd.concat(crosstab_results,axis=1).fillna(0)
crosstab_master = crosstab_master.sort_index(axis=0)

# Save data
crosstab_master.to_csv(CHART_PATH + 'crosstab_master_' + str(datetime.datetime.now().date()).replace('-','') + '.csv')

/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/multi.py:3559: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  result = lib.fast_unique_multiple([self._values, rvals], sort=sort)


In [28]:
print(len(crosstab_master))
print(len(crosstab_master.columns))
crosstab_master.head(2)

910
33


Entity                                                               Accounts  \
Group                                                               Aggregate   
Metric                                                                  Total   
year month has_screen_sess has_appt has_group_res has_content_views             
2020 4     0               0        0             0                    1963.0   
                                                  1                     576.0   

Entity                                                                            \
Group                                                               Account Type   
Metric                                                                 Anonymous   
year month has_screen_sess has_appt has_group_res has_content_views                
2020 4     0               0        0             0                       1302.0   
                                                  1                        287.0   

Entity                                                                        \
Group                                                                          
Metric                                                              Verified   
year month has_screen_sess has_appt has_group_res has_content_views            
2020 4     0               0        0             0                    661.0   
                                                  1                    289.0   

Entity                                                                              \
Group                                                               Account Source   
Metric                                                                   Anonymous   
year month has_screen_sess has_appt has_group_res has_content_views                  
2020 4     0               0        0             0                         1302.0   
                                                  1                          287.0   

Entity                                                                              \
Group                                                                                
Metric                                                              Email_password   
year month has_screen_sess has_appt has_group_res has_content_views                  
2020 4     0               0        0             0                            0.0   
                                                  1                            0.0   

Entity                                                                            \
Group                                                                              
Metric                                                              Penn_key_sso   
year month has_screen_sess has_appt has_group_res has_content_views                
2020 4     0               0        0             0                          4.0   
                                                  1                          1.0   

Entity                                                                            \
Group                                                                              
Metric                                                              Penn_mpm_sso   
year month has_screen_sess has_appt has_group_res has_content_views                
2020 4     0               0        0             0                          0.0   
                                                  1                          0.0   

Entity                                                                        \
Group                                                                          
Metric                                                              Penn_sso   
year month has_screen_sess has_appt has_group_res has_content_views            
2020 4     0               0        0             0                    657.0   
                                                  1                    288.0   

En

# Generate Reports

In [29]:
def get_rel_pct(df,idx=True,idx_lvl=-1,col=True,col_lvl=-1):
    df_dict = {}
    # Index group relative pcts
    if idx:
        idx_levels = df.index.names[:idx_lvl]
        idx_dict = {item:'_'.join([item,report_pct_dict[idx_levels[idx_lvl]],'Pct']) for item in df.columns.get_level_values(idx_lvl)}
        idx_pct = df.groupby(axis=0, level=idx_levels).apply(lambda x: x.div(x.sum(axis=0),axis=1))
        idx_pct = idx_pct.rename(columns=idx_dict)
        # Concat with df and reorder columns
        idx_usecols = [col for item in zip(df.columns,idx_pct.columns) for col in item]
        df_idx_pct = pd.concat([df,idx_pct], axis=1)[idx_usecols]
        try:
            df_idx_pct = df_idx_pct.sort_index(axis=0)
        except:
            print('Could not sort index! Continuing...')
        df_dict['idx'] = df_idx_pct
    
    # Col group relative pcts
    if col:
        col_levels= df.columns.names[:col_lvl]
        col_dict = {item:'_'.join([item,report_pct_dict[col_levels[col_lvl]],'Pct']) for item in df.columns.get_level_values(col_lvl)}
        col_pct = df.groupby(axis=1, level=col_levels).apply(lambda x: x.div(x.sum(axis=1),axis=0))
        col_pct = col_pct.rename(columns=col_dict)
        # Concat with df and reorder columns
        col_usecols = [col for item in zip(df.columns,col_pct.columns) for col in item]
        df_col_pct = pd.concat([df,col_pct], axis=1)[col_usecols]
        try:
            df_col_pct = df_col_pct.sort_index(axis=0)
        except:
            print('Could not sort index! Continuing...')
        df_dict['col'] = df_col_pct
        
    return df_dict

In [30]:
def set_report_group_record(group_name,group_label,group_data=[],save_csv=True):
    report_date = str(datetime.datetime.now().date())
    
    # Set report group record
    group_record = copy.deepcopy(report_group_record)
    group_record['name'] = group_name
    group_record['label'] = group_label
    group_record['date'] = report_date
    
    # Set individual report records
    for df in group_data:
        set_report_record(group_record,df,save_csv)
    
    # Add to report group dict
    report_group_dict[group_name] = group_record
    report_group_dict['group_order'].append(group_name)
    
    return

In [31]:
def set_report_record(group_record,df,save_csv=True):    
    # Define dynamic properties
    data_name = df.name
    data_types = ['freq']
    if 'pct' in data_name.split('_'):
        data_types.append('pct')
    
    # Set record
    record = copy.deepcopy(report_record)
    record['data'] = df
    record['data_name'] = data_name
    record['data_label'] = data_name.replace('_',' ').title()
    record['data_types'] = data_types
    record['idx_group'] = df.index.names
    record['col_group'] = df.columns.names
    
    # Add to group records
    group_record['records'][data_name] = record
    group_record['record_order'].append(data_name)
    
    if save_csv:
        report_date = group_record['date']
        save_date = report_date.replace('-','')
        save_name = REPORT_PATH + data_name + '_' + save_date + '.csv'
        df.to_csv(save_name)
    
    return

In [32]:
idx_slice = pd.IndexSlice

report_group_dict = {'group_order':[]}
report_group_record = {'name':'',
                       'label':'',
                       'date':'',
                       'records':{},
                       'record_order':[]}
report_record = {'data':pd.DataFrame(),
                 'data_name':'',
                 'data_label':'',
                 'data_types':[],
                 'idx_group':[],
                 'col_group':[]}

In [33]:
# Index group labels
report_idx_names = ['year','month','has_screen_sess','has_appt','has_group_res','has_content_views']
report_idx_labels = ['Year','Month','Screening Session','Connect with Support','In the Studio','On Your Time']
report_idx_pct_labels = ['Year','Month','Screening','Support','Studio','Content']
report_idx_dict = dict(zip(report_idx_names,report_idx_labels))

# Column group labels
report_col_names = ['Entity','Group','Metric']
report_col_labels = ['Entity','Group','Metric']
report_col_pct_labels = ['Entity','Group','Metric']
report_col_dict = dict(zip(report_col_names,report_col_labels))

# Relative pct column labels
report_pct_names = report_idx_labels + report_col_labels
report_pct_labels = report_idx_pct_labels + report_col_pct_labels
report_pct_dict = dict(zip(report_pct_names,report_pct_labels))

In [34]:
# Cobalt domain labels
cobalt_domain_names = ['has_appt','has_group_res','has_content_views']
cobalt_domain_labels = ['Connect with Support','In the Studio','On Your Time']
cobalt_domain_dict = dict(zip(cobalt_domain_names,cobalt_domain_labels))

## Master Report

### All Idx Groups

In [35]:
# Master report
master_idx_names = ['year','month','has_screen_sess','has_appt','has_group_res','has_content_views']
master_idx_labels = ['Year','Month','Screening Session','Connect with Support','In the Studio','On Your Time']
master_idx_dict = dict(zip(master_idx_names,master_idx_labels))

# Freq
report_master = crosstab_master.groupby(axis=0,level=[master_idx_names]).sum()
report_master.index = report_master.index.rename(master_idx_dict)
report_master = report_master.sort_index(axis=0)

# Pct
report_master_pct_dict = get_rel_pct(report_master)
report_master_pct_idx = report_master_pct_dict['idx']
report_master_pct_col = report_master_pct_dict['col']

# Set df names
report_master.name = 'report_master'
report_master_pct_idx.name = 'report_master_pct_idx'
report_master_pct_col.name = 'report_master_pct_col'

# Save data
group_name = 'report_master'
group_label = 'Master'
group_data = [report_master,report_master_pct_idx,report_master_pct_col]
set_report_group_record(group_name,group_label,group_data)

In [36]:
print(len(report_master))
print(len(report_master.columns))
report_master.head(2)

910
33


Entity                                                                        Accounts  \
Group                                                                        Aggregate   
Metric                                                                           Total   
Year Month Screening Session Connect with Support In the Studio On Your Time             
2020 4     0                 0                    0             0               1963.0   
                                                                1                576.0   

Entity                                                                                     \
Group                                                                        Account Type   
Metric                                                                          Anonymous   
Year Month Screening Session Connect with Support In the Studio On Your Time                
2020 4     0                 0                    0             0                  1302.0   
                                                                1                   287.0   

Entity                                                                                 \
Group                                                                                   
Metric                                                                       Verified   
Year Month Screening Session Connect with Support In the Studio On Your Time            
2020 4     0                 0                    0             0               661.0   
                                                                1               289.0   

Entity                                                                                       \
Group                                                                        Account Source   
Metric                                                                            Anonymous   
Year Month Screening Session Connect with Support In the Studio On Your Time                  
2020 4     0                 0                    0             0                    1302.0   
                                                                1                     287.0   

Entity                                                                                       \
Group                                                                                         
Metric                                                                       Email_password   
Year Month Screening Session Connect with Support In the Studio On Your Time                  
2020 4     0                 0                    0             0                       0.0   
                                                                1                       0.0   

Entity                                                                                     \
Group                                                                                       
Metric                                                                       Penn_key_sso   
Year Month Screening Session Connect with Support In the Studio On Your Time                
2020 4     0                 0                    0             0                     4.0   
                                                                1                     1.0   

Entity                                                                                     \
Group                                                                                       
Metric                                                                       Penn_mpm_sso   
Year Month Screening Session Connect with Support In the Studio On Your Time                
2020 4     0                 0                    0             0                     0.0   
                                                                1                     0.0   

Entity                                                                                 \
Group           

In [37]:
print(len(report_master_pct_idx))
print(len(report_master_pct_idx.columns))
report_master_pct_idx.head(2)

910
66


Entity                                                                        Accounts  \
Group                                                                        Aggregate   
Metric                                                                           Total   
Year Month Screening Session Connect with Support In the Studio On Your Time             
2020 4     0                 0                    0             0               1963.0   
                                                                1                576.0   

Entity                                                                                         \
Group                                                                                           
Metric                                                                       Total_Studio_Pct   
Year Month Screening Session Connect with Support In the Studio On Your Time                    
2020 4     0                 0                    0             0                      0.7731   
                                                                1                      0.2269   

Entity                                                                                     \
Group                                                                        Account Type   
Metric                                                                          Anonymous   
Year Month Screening Session Connect with Support In the Studio On Your Time                
2020 4     0                 0                    0             0                  1302.0   
                                                                1                   287.0   

Entity                                                                                             \
Group                                                                                               
Metric                                                                       Anonymous_Studio_Pct   
Year Month Screening Session Connect with Support In the Studio On Your Time                        
2020 4     0                 0                    0             0                          0.8194   
                                                                1                          0.1806   

Entity                                                                                 \
Group                                                                                   
Metric                                                                       Verified   
Year Month Screening Session Connect with Support In the Studio On Your Time            
2020 4     0                 0                    0             0               661.0   
                                                                1               289.0   

Entity                                                                                            \
Group                                                                                              
Metric                                                                       Verified_Studio_Pct   
Year Month Screening Session Connect with Support In the Studio On Your Time                       
2020 4     0                 0                    0             0                         0.6958   
                                                                1                         0.3042   

Entity                                                                                       \
Group                                                                        Account Source   
Metric                                                                            Anonymous   
Year Month Screening Session Connect with Support In the Studio On Your Time                  
2020 4     0                 0                    0             0                    1302.0   
                                                                1                     287.0   

Ent

In [38]:
print(len(report_master_pct_col))
print(len(report_master_pct_col.columns))
report_master_pct_col.head(2)

910
66


Entity                                                                        Accounts  \
Group                                                                        Aggregate   
Metric                                                                           Total   
Year Month Screening Session Connect with Support In the Studio On Your Time             
2020 4     0                 0                    0             0               1963.0   
                                                                1                576.0   

Entity                                                                                        \
Group                                                                                          
Metric                                                                       Total_Group_Pct   
Year Month Screening Session Connect with Support In the Studio On Your Time                   
2020 4     0                 0                    0             0                        1.0   
                                                                1                        1.0   

Entity                                                                                     \
Group                                                                        Account Type   
Metric                                                                          Anonymous   
Year Month Screening Session Connect with Support In the Studio On Your Time                
2020 4     0                 0                    0             0                  1302.0   
                                                                1                   287.0   

Entity                                                                                            \
Group                                                                                              
Metric                                                                       Anonymous_Group_Pct   
Year Month Screening Session Connect with Support In the Studio On Your Time                       
2020 4     0                 0                    0             0                         0.6633   
                                                                1                         0.4983   

Entity                                                                                 \
Group                                                                                   
Metric                                                                       Verified   
Year Month Screening Session Connect with Support In the Studio On Your Time            
2020 4     0                 0                    0             0               661.0   
                                                                1               289.0   

Entity                                                                                           \
Group                                                                                             
Metric                                                                       Verified_Group_Pct   
Year Month Screening Session Connect with Support In the Studio On Your Time                      
2020 4     0                 0                    0             0                        0.3367   
                                                                1                        0.5017   

Entity                                                                                       \
Group                                                                        Account Source   
Metric                                                                            Anonymous   
Year Month Screening Session Connect with Support In the Studio On Your Time                  
2020 4     0                 0                    0             0                    1302.0   
                                                                1                     287.0   

Entity               

## Yearly Reports

### All Idx Groups

In [39]:
# Yearly report
year_idx_names = ['year','has_screen_sess','has_appt','has_group_res','has_content_views']
year_idx_labels = ['Year','Screening Session','Connect with Support','In the Studio','On Your Time']
year_idx_dict = dict(zip(year_idx_names,year_idx_labels))

# Freq
report_year = crosstab_master.groupby(axis=0,level=[year_idx_names]).sum()
report_year.index = report_year.index.rename(year_idx_dict)
report_year = report_year.sort_index(axis=0)

# Pct
report_year_pct_dict = get_rel_pct(report_year)
report_year_pct_idx = report_year_pct_dict['idx']
report_year_pct_col = report_year_pct_dict['col']

# Set df names
report_year.name = 'report_year'
report_year_pct_idx.name = 'report_year_pct_idx'
report_year_pct_col.name = 'report_year_pct_col'

# Save data
group_name = 'report_year'
group_label = 'Year - Master'
group_data = [report_year,report_year_pct_idx,report_year_pct_col]
set_report_group_record(group_name,group_label,group_data)

In [40]:
print(len(report_year))
print(len(report_year.columns))
report_year.head(2)

94
33


Entity                                                                  Accounts  \
Group                                                                  Aggregate   
Metric                                                                     Total   
Year Screening Session Connect with Support In the Studio On Your Time             
2020 0                 0                    0             0               9684.0   
                                                          1               1929.0   

Entity                                                                               \
Group                                                                  Account Type   
Metric                                                                    Anonymous   
Year Screening Session Connect with Support In the Studio On Your Time                
2020 0                 0                    0             0                  7551.0   
                                                          1                  1291.0   

Entity                                                                           \
Group                                                                             
Metric                                                                 Verified   
Year Screening Session Connect with Support In the Studio On Your Time            
2020 0                 0                    0             0              2133.0   
                                                          1               638.0   

Entity                                                                                 \
Group                                                                  Account Source   
Metric                                                                      Anonymous   
Year Screening Session Connect with Support In the Studio On Your Time                  
2020 0                 0                    0             0                    7551.0   
                                                          1                    1291.0   

Entity                                                                                 \
Group                                                                                   
Metric                                                                 Email_password   
Year Screening Session Connect with Support In the Studio On Your Time                  
2020 0                 0                    0             0                       1.0   
                                                          1                       0.0   

Entity                                                                               \
Group                                                                                 
Metric                                                                 Penn_key_sso   
Year Screening Session Connect with Support In the Studio On Your Time                
2020 0                 0                    0             0                   337.0   
                                                          1                    52.0   

Entity                                                                               \
Group                                                                                 
Metric                                                                 Penn_mpm_sso   
Year Screening Session Connect with Support In the Studio On Your Time                
2020 0                 0                    0             0                     0.0   
                                                          1                     0.0   

Entity                                                                           \
Group                                                                             
Metric                                                                 Penn_sso   
Year Screening Session Connect with Support In the Studio On Your Time            
2020 0                 0 

In [41]:
print(len(report_year_pct_idx))
print(len(report_year_pct_idx.columns))
report_year_pct_idx.head(2)

94
66


Entity                                                                  Accounts  \
Group                                                                  Aggregate   
Metric                                                                     Total   
Year Screening Session Connect with Support In the Studio On Your Time             
2020 0                 0                    0             0               9684.0   
                                                          1               1929.0   

Entity                                                                                   \
Group                                                                                     
Metric                                                                 Total_Studio_Pct   
Year Screening Session Connect with Support In the Studio On Your Time                    
2020 0                 0                    0             0                      0.8339   
                                                          1                      0.1661   

Entity                                                                               \
Group                                                                  Account Type   
Metric                                                                    Anonymous   
Year Screening Session Connect with Support In the Studio On Your Time                
2020 0                 0                    0             0                  7551.0   
                                                          1                  1291.0   

Entity                                                                                       \
Group                                                                                         
Metric                                                                 Anonymous_Studio_Pct   
Year Screening Session Connect with Support In the Studio On Your Time                        
2020 0                 0                    0             0                           0.854   
                                                          1                           0.146   

Entity                                                                           \
Group                                                                             
Metric                                                                 Verified   
Year Screening Session Connect with Support In the Studio On Your Time            
2020 0                 0                    0             0              2133.0   
                                                          1               638.0   

Entity                                                                                      \
Group                                                                                        
Metric                                                                 Verified_Studio_Pct   
Year Screening Session Connect with Support In the Studio On Your Time                       
2020 0                 0                    0             0                         0.7698   
                                                          1                         0.2302   

Entity                                                                                 \
Group                                                                  Account Source   
Metric                                                                      Anonymous   
Year Screening Session Connect with Support In the Studio On Your Time                  
2020 0                 0                    0             0                    7551.0   
                                                          1                    1291.0   

Entity                                                                                       \
Group                                                                                         
Metric                                                           

In [42]:
print(len(report_year_pct_col))
print(len(report_year_pct_col.columns))
report_year_pct_col.head(2)

94
66


Entity                                                                  Accounts  \
Group                                                                  Aggregate   
Metric                                                                     Total   
Year Screening Session Connect with Support In the Studio On Your Time             
2020 0                 0                    0             0               9684.0   
                                                          1               1929.0   

Entity                                                                                  \
Group                                                                                    
Metric                                                                 Total_Group_Pct   
Year Screening Session Connect with Support In the Studio On Your Time                   
2020 0                 0                    0             0                        1.0   
                                                          1                        1.0   

Entity                                                                               \
Group                                                                  Account Type   
Metric                                                                    Anonymous   
Year Screening Session Connect with Support In the Studio On Your Time                
2020 0                 0                    0             0                  7551.0   
                                                          1                  1291.0   

Entity                                                                                      \
Group                                                                                        
Metric                                                                 Anonymous_Group_Pct   
Year Screening Session Connect with Support In the Studio On Your Time                       
2020 0                 0                    0             0                         0.7797   
                                                          1                         0.6693   

Entity                                                                           \
Group                                                                             
Metric                                                                 Verified   
Year Screening Session Connect with Support In the Studio On Your Time            
2020 0                 0                    0             0              2133.0   
                                                          1               638.0   

Entity                                                                                     \
Group                                                                                       
Metric                                                                 Verified_Group_Pct   
Year Screening Session Connect with Support In the Studio On Your Time                      
2020 0                 0                    0             0                        0.2203   
                                                          1                        0.3307   

Entity                                                                                 \
Group                                                                  Account Source   
Metric                                                                      Anonymous   
Year Screening Session Connect with Support In the Studio On Your Time                  
2020 0                 0                    0             0                    7551.0   
                                                          1                    1291.0   

Entity                                                                                      \
Group                                                                                        
Metric                                                                 Anonymous_Grou

### Aggregate

In [43]:
# Yearly aggregate report

# Freq
report_year_agg = report_year.groupby(axis=0,level='Year').sum()

# Pct
report_year_agg_pct_dict = get_rel_pct(report_year_agg, idx=False)
report_year_agg_pct_col = report_year_agg_pct_dict['col']

# Set df names
report_year_agg.name = 'report_year_agg'
report_year_agg_pct_col.name = 'report_year_agg_pct_col'

# Save data
group_name = 'report_year_agg'
group_label = 'Year - Aggregate'
group_data = [report_year_agg,report_year_agg_pct_col]
set_report_group_record(group_name,group_label,group_data)

Could not sort index! Continuing...


In [44]:
print(len(report_year_agg))
print(len(report_year_agg.columns))
report_year_agg.head(2)

6
33


Entity  Accounts                                                      \
Group  Aggregate Account Type          Account Source                  
Metric     Total    Anonymous Verified      Anonymous Email_password   
Year                                                                   
2020     13543.0       9485.0   4058.0         9485.0            1.0   
2021     12591.0      10200.0   2391.0        10200.0            0.0   

Entity                                    Screening Sessions               \
Group                                              Aggregate Triage Level   
Metric Penn_key_sso Penn_mpm_sso Penn_sso              Total         Mild   
Year                                                                        
2020          542.0          0.0   3515.0             1027.0        441.0   
2021         1187.0         88.0   1116.0              819.0        381.0   

Entity                                    Appointments                 \
Group                  Crisis Flag           Aggregate Clinical Level   
Metric Moderate Severe          No    Yes        Total       Clinical   
Year                                                                    
2020      488.0   98.0       908.0  119.0       1438.0         1066.0   
2021      362.0   76.0       719.0  100.0       1491.0         1039.0   

Entity                                                                   \
Group               Provider Role                                         
Metric Sub-clinical  Care_manager Chaplain Clinician  Coach Other  Peer   
Year                                                                      
2020          372.0           2.0     18.0     794.0  341.0   1.0  10.0   
2021          452.0         160.0      9.0     871.0  282.0   0.0   1.0   

Entity                                     Group Sessions Content Views  \
Group                                           Aggregate     Aggregate   
Metric Psychiatrist Psychologist Undefined          Total         Total   
Year                                                                      
2020          272.0          0.0       0.0           11.0        5053.0   
2021          168.0          0.0       0.0         1255.0        4595.0   

Entity                                                 
Group  Content Type                                    
Metric          App Article Podcast   Video Worksheet  
Year                                                   
2020            0.0  2289.0   872.0  1652.0     240.0  
2021            0.0  1215.0  1988.0  1372.0      20.0

In [45]:
print(len(report_year_agg_pct_col))
print(len(report_year_agg_pct_col.columns))
report_year_agg_pct_col.head(2)

6
66


Entity  Accounts                                                            \
Group  Aggregate                 Account Type                                
Metric     Total Total_Group_Pct    Anonymous Anonymous_Group_Pct Verified   
Year                                                                         
2020     13543.0             1.0       9485.0              0.7004   4058.0   
2021     12591.0             1.0      10200.0              0.8101   2391.0   

Entity                                                                       \
Group                     Account Source                                      
Metric Verified_Group_Pct      Anonymous Anonymous_Group_Pct Email_password   
Year                                                                          
2020               0.2996         9485.0              0.7004            1.0   
2021               0.1899        10200.0              0.8101            0.0   

Entity                                                               \
Group                                                                 
Metric Email_password_Group_Pct Penn_key_sso Penn_key_sso_Group_Pct   
Year                                                                  
2020                 7.3839e-05        542.0                 0.0400   
2021                 0.0000e+00       1187.0                 0.0943   

Entity                                                                  \
Group                                                                    
Metric Penn_mpm_sso Penn_mpm_sso_Group_Pct Penn_sso Penn_sso_Group_Pct   
Year                                                                     
2020            0.0                  0.000   3515.0             0.2595   
2021           88.0                  0.007   1116.0             0.0886   

Entity Screening Sessions                                              \
Group           Aggregate                 Triage Level                  
Metric              Total Total_Group_Pct         Mild Mild_Group_Pct   
Year                                                                    
2020               1027.0             1.0        441.0         0.4294   
2021                819.0             1.0        381.0         0.4652   

Entity                                                                  \
Group                                                      Crisis Flag   
Metric Moderate Moderate_Group_Pct Severe Severe_Group_Pct          No   
Year                                                                     
2020      488.0             0.4752   98.0           0.0954       908.0   
2021      362.0             0.4420   76.0           0.0928       719.0   

Entity                                   Appointments                  \
Group                                       Aggregate                   
Metric No_Group_Pct    Yes Yes_Group_Pct        Total Total_Group_Pct   
Year                                                                    
2020         0.8841  119.0        0.1159       1438.0             1.0   
2021         0.8779  100.0        0.1221       1491.0             1.0   

Entity                                                                        \
Group  Clinical Level                                                          
Metric       Clinical Clinical_Group_Pct Sub-clinical Sub-clinical_Group_Pct   
Year                                                                           
2020           1066.0             0.7413        372.0                 0.2587   
2021           1039.0             0.6968        452.0                 0.3032   

Entity                                                                   \
Group  Provider Role                                                      
Metric  Care_manager Care_manager_Group_Pct Chaplain Chaplain_Group_Pct   
Year                                                                      
2020             2.0                 0.0014     18.0             0.0125   
2021 

### Cobalt Domains

#### All Accounts

In [46]:
# "Cobalt domains" yearly report
domain_idx_names = ['year','has_appt','has_group_res','has_content_views']
domain_idx_labels = ['Year','Connect with Support','In the Studio','On Your Time']
domain_idx_dict = dict(zip(domain_idx_names,domain_idx_labels))

# Freq
report_year_domains = crosstab_master.groupby(axis=0,level=[domain_idx_names]).sum()
report_year_domains.index = report_year_domains.index.rename(domain_idx_dict)
report_year_domains = report_year_domains.sort_index(axis=0)

# Pct
report_year_domains_pct_dict = get_rel_pct(report_year_domains)
report_year_domains_pct_idx = report_year_domains_pct_dict['idx']
report_year_domains_pct_col = report_year_domains_pct_dict['col']

# Set df names
report_year_domains.name = 'report_year_domains'
report_year_domains_pct_idx.name = 'report_year_domains_pct_idx'
report_year_domains_pct_col.name = 'report_year_domains_pct_col'

# Save data
group_name = 'report_year_domains'
group_label = 'Year - Domain'
group_data = [report_year_domains,report_year_domains_pct_idx,report_year_domains_pct_col]
set_report_group_record(group_name,group_label,group_data)

In [47]:
print(len(report_year_domains))
print(len(report_year_domains.columns))
report_year_domains.head(2)

48
33


Entity                                                Accounts               \
Group                                                Aggregate Account Type   
Metric                                                   Total    Anonymous   
Year Connect with Support In the Studio On Your Time                          
2020 0                    0             0              10100.0       7741.0   
                                        1               2029.0       1333.0   

Entity                                                                        \
Group                                                         Account Source   
Metric                                               Verified      Anonymous   
Year Connect with Support In the Studio On Your Time                           
2020 0                    0             0              2359.0         7741.0   
                                        1               696.0         1333.0   

Entity                                                               \
Group                                                                 
Metric                                               Email_password   
Year Connect with Support In the Studio On Your Time                  
2020 0                    0             0                       1.0   
                                        1                       0.0   

Entity                                                             \
Group                                                               
Metric                                               Penn_key_sso   
Year Connect with Support In the Studio On Your Time                
2020 0                    0             0                   378.0   
                                        1                    58.0   

Entity                                                                      \
Group                                                                        
Metric                                               Penn_mpm_sso Penn_sso   
Year Connect with Support In the Studio On Your Time                         
2020 0                    0             0                     0.0   1980.0   
                                        1                     0.0    638.0   

Entity                                               Screening Sessions  \
Group                                                         Aggregate   
Metric                                                            Total   
Year Connect with Support In the Studio On Your Time                      
2020 0                    0             0                         387.0   
                                        1                          84.0   

Entity                                                                      \
Group                                                Triage Level            
Metric                                                       Mild Moderate   
Year Connect with Support In the Studio On Your Time                         
2020 0                    0             0                   200.0    151.0   
                                        1                    50.0     26.0   

Entity                                                                         \
Group                                                       Crisis Flag         
Metric                                               Severe          No   Yes   
Year Connect with Support In the Studio On Your Time                            
2020 0                    0             0              36.0       339.0  48.0   
                                        1               8.0        73.0  11.0   

Entity                                               Appointments  \
Group                                                   Aggregate   
Metric                                                      Total   
Year Connect with Support In the Studio On Your Time                
2020 0                    0             0         

In [48]:
print(len(report_year_domains_pct_idx))
print(len(report_year_domains_pct_idx.columns))
report_year_domains_pct_idx.head(2)

48
66


Entity                                                Accounts  \
Group                                                Aggregate   
Metric                                                   Total   
Year Connect with Support In the Studio On Your Time             
2020 0                    0             0              10100.0   
                                        1               2029.0   

Entity                                                                 \
Group                                                                   
Metric                                               Total_Studio_Pct   
Year Connect with Support In the Studio On Your Time                    
2020 0                    0             0                      0.8327   
                                        1                      0.1673   

Entity                                                             \
Group                                                Account Type   
Metric                                                  Anonymous   
Year Connect with Support In the Studio On Your Time                
2020 0                    0             0                  7741.0   
                                        1                  1333.0   

Entity                                                                     \
Group                                                                       
Metric                                               Anonymous_Studio_Pct   
Year Connect with Support In the Studio On Your Time                        
2020 0                    0             0                          0.8531   
                                        1                          0.1469   

Entity                                                         \
Group                                                           
Metric                                               Verified   
Year Connect with Support In the Studio On Your Time            
2020 0                    0             0              2359.0   
                                        1               696.0   

Entity                                                                    \
Group                                                                      
Metric                                               Verified_Studio_Pct   
Year Connect with Support In the Studio On Your Time                       
2020 0                    0             0                         0.7722   
                                        1                         0.2278   

Entity                                                               \
Group                                                Account Source   
Metric                                                    Anonymous   
Year Connect with Support In the Studio On Your Time                  
2020 0                    0             0                    7741.0   
                                        1                    1333.0   

Entity                                                                     \
Group                                                                       
Metric                                               Anonymous_Studio_Pct   
Year Connect with Support In the Studio On Your Time                        
2020 0                    0             0                          0.8531   
                                        1                          0.1469   

Entity                                                               \
Group                                                                 
Metric                                               Email_password   
Year Connect with Support In the Studio On Your Time                  
2020 0                    0             0                       1.0   
                                        1                       0.0   

Entity                                                                          \
Group                                  

In [49]:
print(len(report_year_domains_pct_col))
print(len(report_year_domains_pct_col.columns))
report_year_domains_pct_col.head(2)

48
66


Entity                                                Accounts  \
Group                                                Aggregate   
Metric                                                   Total   
Year Connect with Support In the Studio On Your Time             
2020 0                    0             0              10100.0   
                                        1               2029.0   

Entity                                                                \
Group                                                                  
Metric                                               Total_Group_Pct   
Year Connect with Support In the Studio On Your Time                   
2020 0                    0             0                        1.0   
                                        1                        1.0   

Entity                                                             \
Group                                                Account Type   
Metric                                                  Anonymous   
Year Connect with Support In the Studio On Your Time                
2020 0                    0             0                  7741.0   
                                        1                  1333.0   

Entity                                                                    \
Group                                                                      
Metric                                               Anonymous_Group_Pct   
Year Connect with Support In the Studio On Your Time                       
2020 0                    0             0                         0.7664   
                                        1                         0.6570   

Entity                                                         \
Group                                                           
Metric                                               Verified   
Year Connect with Support In the Studio On Your Time            
2020 0                    0             0              2359.0   
                                        1               696.0   

Entity                                                                   \
Group                                                                     
Metric                                               Verified_Group_Pct   
Year Connect with Support In the Studio On Your Time                      
2020 0                    0             0                        0.2336   
                                        1                        0.3430   

Entity                                                               \
Group                                                Account Source   
Metric                                                    Anonymous   
Year Connect with Support In the Studio On Your Time                  
2020 0                    0             0                    7741.0   
                                        1                    1333.0   

Entity                                                                    \
Group                                                                      
Metric                                               Anonymous_Group_Pct   
Year Connect with Support In the Studio On Your Time                       
2020 0                    0             0                         0.7664   
                                        1                         0.6570   

Entity                                                               \
Group                                                                 
Metric                                               Email_password   
Year Connect with Support In the Studio On Your Time                  
2020 0                    0             0                       1.0   
                                        1                       0.0   

Entity                                                                         \
Group                                                           

#### Domain Activity

In [50]:
# "Cobalt domains" yearly report - active accounts
# Aggregate individual domain data
temp_data = []
num_levels = len(report_year_domains.index.levels)
cobalt_domains = [item for item in enumerate(report_year_domains.index.names) if item[1] in cobalt_domain_labels]
for domain_idx,domain_name in cobalt_domains:
    temp_idx_slice = tuple(1 if item==domain_idx else slice(None, None, None) for item in range(num_levels))
    temp_df = report_year_domains.loc[temp_idx_slice,:].groupby(axis=0, level=['Year',domain_idx]).sum()
    temp_df = temp_df.rename(index={1:domain_name},level=1)
    temp_df.index = temp_df.index.rename(['Year','Domain'])
    temp_data.append(temp_df)    

# Aggregate site-wide data
site_wide = report_year_domains.groupby(axis=0, level='Year').sum()
site_wide.index = pd.MultiIndex.from_product([site_wide.index, ['Site_wide']], names=['Year','Domain'])
temp_data.append(site_wide)

# Freq
report_year_domains_active = pd.concat(temp_data).sort_index()

# Pct
report_year_domains_active_pct_dict = get_rel_pct(report_year_domains_active)
report_year_domains_active_pct_idx = report_year_domains_active_pct_dict['idx']
report_year_domains_active_pct_col = report_year_domains_active_pct_dict['col']

# Set df names
report_year_domains_active.name = 'report_year_domains_active'
report_year_domains_active_pct_idx.name = 'report_year_domains_active_pct_idx'
report_year_domains_active_pct_col.name = 'report_year_domains_active_pct_col'

# Save data
group_name = 'report_year_domains_active'
group_label = 'Year - Domain Activity'
group_data = [report_year_domains_active,report_year_domains_active_pct_idx,report_year_domains_active_pct_col]
set_report_group_record(group_name,group_label,group_data)

In [51]:
print(len(report_year_domains_active))
print(len(report_year_domains_active.columns))
report_year_domains_active.head(2)

24
33


Entity                     Accounts                                       \
Group                     Aggregate Account Type          Account Source   
Metric                        Total    Anonymous Verified      Anonymous   
Year Domain                                                                
2020 Connect with Support     960.0        373.0    587.0          373.0   
     In the Studio            566.0         46.0    520.0           46.0   

Entity                                                                       \
Group                                                                         
Metric                    Email_password Penn_key_sso Penn_mpm_sso Penn_sso   
Year Domain                                                                   
2020 Connect with Support            0.0         72.0          0.0    515.0   
     In the Studio                   0.0         45.0          0.0    475.0   

Entity                    Screening Sessions                               \
Group                              Aggregate Triage Level                   
Metric                                 Total         Mild Moderate Severe   
Year Domain                                                                 
2020 Connect with Support              520.0        165.0    302.0   53.0   
     In the Studio                      89.0         47.0     40.0    2.0   

Entity                                      Appointments                 \
Group                     Crisis Flag          Aggregate Clinical Level   
Metric                             No   Yes        Total       Clinical   
Year Domain                                                               
2020 Connect with Support       462.0  58.0       1438.0         1066.0   
     In the Studio               83.0   6.0        116.0           83.0   

Entity                                                                   \
Group                                  Provider Role                      
Metric                    Sub-clinical  Care_manager Chaplain Clinician   
Year Domain                                                               
2020 Connect with Support        372.0           2.0     18.0     794.0   
     In the Studio                33.0           0.0      4.0      71.0   

Entity                                                                  \
Group                                                                    
Metric                     Coach Other  Peer Psychiatrist Psychologist   
Year Domain                                                              
2020 Connect with Support  341.0   1.0  10.0        272.0          0.0   
     In the Studio          29.0   0.0   0.0         12.0          0.0   

Entity                              Group Sessions Content Views               \
Group                                    Aggregate     Aggregate Content Type   
Metric                    Undefined          Total         Total          App   
Year Domain                                                                     
2020 Connect with Support       0.0            7.0         581.0          0.0   
     In the Studio              0.0           11.0         602.0          0.0   

Entity                                                      
Group                                                       
Metric                    Article Podcast  Video Worksheet  
Year Domain                                                 
2020 Connect with Support   291.0    96.0  159.0      35.0  
     In the Studio          296.0    68.0  204.0      34.0

In [52]:
print(len(report_year_domains_active_pct_idx))
print(len(report_year_domains_active_pct_idx.columns))
report_year_domains_active_pct_idx.head(2)

24
66


Entity                     Accounts                              \
Group                     Aggregate                Account Type   
Metric                        Total Total_Year_Pct    Anonymous   
Year Domain                                                       
2020 Connect with Support     960.0         0.0548        373.0   
     In the Studio            566.0         0.0323         46.0   

Entity                                                                   \
Group                                                                     
Metric                    Anonymous_Year_Pct Verified Verified_Year_Pct   
Year Domain                                                               
2020 Connect with Support             0.0330    587.0            0.0946   
     In the Studio                    0.0041    520.0            0.0838   

Entity                                                                      \
Group                     Account Source                                     
Metric                         Anonymous Anonymous_Year_Pct Email_password   
Year Domain                                                                  
2020 Connect with Support          373.0             0.0330            0.0   
     In the Studio                  46.0             0.0041            0.0   

Entity                                                          \
Group                                                            
Metric                    Email_password_Year_Pct Penn_key_sso   
Year Domain                                                      
2020 Connect with Support                     0.0         72.0   
     In the Studio                            0.0         45.0   

Entity                                                        \
Group                                                          
Metric                    Penn_key_sso_Year_Pct Penn_mpm_sso   
Year Domain                                                    
2020 Connect with Support                0.0972          0.0   
     In the Studio                       0.0607          0.0   

Entity                                                                      \
Group                                                                        
Metric                    Penn_mpm_sso_Year_Pct Penn_sso Penn_sso_Year_Pct   
Year Domain                                                                  
2020 Connect with Support                   NaN    515.0            0.0943   
     In the Studio                          NaN    475.0            0.0870   

Entity                    Screening Sessions                              \
Group                              Aggregate                Triage Level   
Metric                                 Total Total_Year_Pct         Mild   
Year Domain                                                                
2020 Connect with Support              520.0         0.2811        165.0   
     In the Studio                      89.0         0.0481         47.0   

Entity                                                                     \
Group                                                                       
Metric                    Mild_Year_Pct Moderate Moderate_Year_Pct Severe   
Year Domain                                                                 
2020 Connect with Support        0.2191    302.0            0.3240   53.0   
     In the Studio               0.0624     40.0            0.0429    2.0   

Entity                                                                   \
Group                                     Crisis Flag                     
Metric                    Severe_Year_Pct          No No_Year_Pct   Yes   
Year Domain                                                               
2020 Connect with Support          0.3212       462.0      0.2807  58.0   
     In the Studio                 0.0121        83.0      0.0504   6.0   

Entity                                 Appointments    

In [53]:
print(len(report_year_domains_active_pct_col))
print(len(report_year_domains_active_pct_col.columns))
report_year_domains_active_pct_col.head(2)

24
66


Entity                     Accounts                               \
Group                     Aggregate                 Account Type   
Metric                        Total Total_Group_Pct    Anonymous   
Year Domain                                                        
2020 Connect with Support     960.0             1.0        373.0   
     In the Studio            566.0             1.0         46.0   

Entity                                                                     \
Group                                                                       
Metric                    Anonymous_Group_Pct Verified Verified_Group_Pct   
Year Domain                                                                 
2020 Connect with Support              0.3885    587.0             0.6115   
     In the Studio                     0.0813    520.0             0.9187   

Entity                                                                       \
Group                     Account Source                                      
Metric                         Anonymous Anonymous_Group_Pct Email_password   
Year Domain                                                                   
2020 Connect with Support          373.0              0.3885            0.0   
     In the Studio                  46.0              0.0813            0.0   

Entity                                                           \
Group                                                             
Metric                    Email_password_Group_Pct Penn_key_sso   
Year Domain                                                       
2020 Connect with Support                      0.0         72.0   
     In the Studio                             0.0         45.0   

Entity                                                         \
Group                                                           
Metric                    Penn_key_sso_Group_Pct Penn_mpm_sso   
Year Domain                                                     
2020 Connect with Support                 0.0750          0.0   
     In the Studio                        0.0795          0.0   

Entity                                                                        \
Group                                                                          
Metric                    Penn_mpm_sso_Group_Pct Penn_sso Penn_sso_Group_Pct   
Year Domain                                                                    
2020 Connect with Support                    0.0    515.0             0.5365   
     In the Studio                           0.0    475.0             0.8392   

Entity                    Screening Sessions                               \
Group                              Aggregate                 Triage Level   
Metric                                 Total Total_Group_Pct         Mild   
Year Domain                                                                 
2020 Connect with Support              520.0             1.0        165.0   
     In the Studio                      89.0             1.0         47.0   

Entity                                                                       \
Group                                                                         
Metric                    Mild_Group_Pct Moderate Moderate_Group_Pct Severe   
Year Domain                                                                   
2020 Connect with Support         0.3173    302.0             0.5808   53.0   
     In the Studio                0.5281     40.0             0.4494    2.0   

Entity                                                                     \
Group                                      Crisis Flag                      
Metric                    Severe_Group_Pct          No No_Group_Pct   Yes   
Year Domain                                                                 
2020 Connect with Support           0.1019       462.0       0.8885  58.0   
     In the Studio                  0.0225        83.0 

#### Active Accounts
* Any account that has utilized at least one cobalt resource - appts, group sessions, or content

In [54]:
# Active accounts yearly report
exclude_idx = [item for item in report_year_domains.index if sum(item[1:])==0]

# Freq
report_year_active_accounts = report_year_domains.loc[~report_year_domains.index.isin(exclude_idx)].copy()
report_year_active_accounts = report_year_active_accounts.groupby(axis=0, level='Year').sum()

# Pct
report_year_active_accounts_pct_dict = get_rel_pct(report_year_active_accounts,idx=False)
report_year_active_accounts_pct_col = report_year_active_accounts_pct_dict['col']

# Set df names
report_year_active_accounts.name = 'report_year_active_accounts'
report_year_active_accounts_pct_col.name = 'report_year_active_accounts_pct_col'

# Save data
group_name = 'report_year_active_accounts'
group_label = 'Year - Active Accounts'
group_data = [report_year_active_accounts,report_year_active_accounts_pct_col]
set_report_group_record(group_name,group_label,group_data)

Could not sort index! Continuing...


In [55]:
print(len(report_year_active_accounts))
print(len(report_year_active_accounts.columns))
report_year_active_accounts.head(2)

6
33


Entity  Accounts                                                      \
Group  Aggregate Account Type          Account Source                  
Metric     Total    Anonymous Verified      Anonymous Email_password   
Year                                                                   
2020      3443.0       1744.0   1699.0         1744.0            0.0   
2021      3137.0       2000.0   1137.0         2000.0            0.0   

Entity                                    Screening Sessions               \
Group                                              Aggregate Triage Level   
Metric Penn_key_sso Penn_mpm_sso Penn_sso              Total         Mild   
Year                                                                        
2020          164.0          0.0   1535.0              640.0        241.0   
2021          593.0          0.0    544.0              472.0        210.0   

Entity                                   Appointments                 \
Group                  Crisis Flag          Aggregate Clinical Level   
Metric Moderate Severe          No   Yes        Total       Clinical   
Year                                                                   
2020      337.0   62.0       569.0  71.0       1438.0         1066.0   
2021      221.0   41.0       416.0  56.0       1491.0         1039.0   

Entity                                                                   \
Group               Provider Role                                         
Metric Sub-clinical  Care_manager Chaplain Clinician  Coach Other  Peer   
Year                                                                      
2020          372.0           2.0     18.0     794.0  341.0   1.0  10.0   
2021          452.0         160.0      9.0     871.0  282.0   0.0   1.0   

Entity                                     Group Sessions Content Views  \
Group                                           Aggregate     Aggregate   
Metric Psychiatrist Psychologist Undefined          Total         Total   
Year                                                                      
2020          272.0          0.0       0.0           11.0        5053.0   
2021          168.0          0.0       0.0         1255.0        4595.0   

Entity                                                 
Group  Content Type                                    
Metric          App Article Podcast   Video Worksheet  
Year                                                   
2020            0.0  2289.0   872.0  1652.0     240.0  
2021            0.0  1215.0  1988.0  1372.0      20.0

In [56]:
print(len(report_year_active_accounts_pct_col))
print(len(report_year_active_accounts_pct_col.columns))
report_year_active_accounts_pct_col.head(2)

6
66


Entity  Accounts                                                            \
Group  Aggregate                 Account Type                                
Metric     Total Total_Group_Pct    Anonymous Anonymous_Group_Pct Verified   
Year                                                                         
2020      3443.0             1.0       1744.0              0.5065   1699.0   
2021      3137.0             1.0       2000.0              0.6376   1137.0   

Entity                                                                       \
Group                     Account Source                                      
Metric Verified_Group_Pct      Anonymous Anonymous_Group_Pct Email_password   
Year                                                                          
2020               0.4935         1744.0              0.5065            0.0   
2021               0.3624         2000.0              0.6376            0.0   

Entity                                                               \
Group                                                                 
Metric Email_password_Group_Pct Penn_key_sso Penn_key_sso_Group_Pct   
Year                                                                  
2020                        0.0        164.0                 0.0476   
2021                        0.0        593.0                 0.1890   

Entity                                                                  \
Group                                                                    
Metric Penn_mpm_sso Penn_mpm_sso_Group_Pct Penn_sso Penn_sso_Group_Pct   
Year                                                                     
2020            0.0                    0.0   1535.0             0.4458   
2021            0.0                    0.0    544.0             0.1734   

Entity Screening Sessions                                              \
Group           Aggregate                 Triage Level                  
Metric              Total Total_Group_Pct         Mild Mild_Group_Pct   
Year                                                                    
2020                640.0             1.0        241.0         0.3766   
2021                472.0             1.0        210.0         0.4449   

Entity                                                                  \
Group                                                      Crisis Flag   
Metric Moderate Moderate_Group_Pct Severe Severe_Group_Pct          No   
Year                                                                     
2020      337.0             0.5266   62.0           0.0969       569.0   
2021      221.0             0.4682   41.0           0.0869       416.0   

Entity                                  Appointments                  \
Group                                      Aggregate                   
Metric No_Group_Pct   Yes Yes_Group_Pct        Total Total_Group_Pct   
Year                                                                   
2020         0.8891  71.0        0.1109       1438.0             1.0   
2021         0.8814  56.0        0.1186       1491.0             1.0   

Entity                                                                        \
Group  Clinical Level                                                          
Metric       Clinical Clinical_Group_Pct Sub-clinical Sub-clinical_Group_Pct   
Year                                                                           
2020           1066.0             0.7413        372.0                 0.2587   
2021           1039.0             0.6968        452.0                 0.3032   

Entity                                                                   \
Group  Provider Role                                                      
Metric  Care_manager Care_manager_Group_Pct Chaplain Chaplain_Group_Pct   
Year                                                                      
2020             2.0                 0.0014     18.0             0.0125   
2021       

### Screenings

#### Screenings

In [57]:
# Screening and appointments yearly report
screen_idx_names = ['year','has_screen_sess']
screen_idx_labels = ['Year','Screening Session']
screen_idx_dict = dict(zip(screen_idx_names,screen_idx_labels))

# Freq
report_year_screen = crosstab_master.groupby(axis=0,level=[screen_idx_names]).sum()
report_year_screen.index = report_year_screen.index.rename(screen_idx_dict)
report_year_screen = report_year_screen.sort_index(axis=0)

# Pct
report_year_screen_pct_dict = get_rel_pct(report_year_screen)
report_year_screen_pct_idx = report_year_screen_pct_dict['idx']
report_year_screen_pct_col = report_year_screen_pct_dict['col']

# Set df names
report_year_screen.name = 'report_year_screen'
report_year_screen_pct_idx.name = 'report_year_screen_pct_idx'
report_year_screen_pct_col.name = 'report_year_screen_pct_col'

# Save data
group_name = 'report_year_screen'
group_label = 'Year - Screen'
group_data = [report_year_screen,report_year_screen_pct_idx,report_year_screen_pct_col]
set_report_group_record(group_name,group_label,group_data)

In [58]:
print(len(report_year_screen))
print(len(report_year_screen.columns))
report_year_screen.head(2)

12
33


Entity                  Accounts                                       \
Group                  Aggregate Account Type          Account Source   
Metric                     Total    Anonymous Verified      Anonymous   
Year Screening Session                                                  
2020 0                   12375.0       9129.0   3246.0         9129.0   
     1                    1168.0        356.0    812.0          356.0   

Entity                                                                    \
Group                                                                      
Metric                 Email_password Penn_key_sso Penn_mpm_sso Penn_sso   
Year Screening Session                                                     
2020 0                            1.0        430.0          0.0   2815.0   
     1                            0.0        112.0          0.0    700.0   

Entity                 Screening Sessions                               \
Group                           Aggregate Triage Level                   
Metric                              Total         Mild Moderate Severe   
Year Screening Session                                                   
2020 0                                0.0          0.0      0.0    0.0   
     1                             1027.0        441.0    488.0   98.0   

Entity                                    Appointments                 \
Group                  Crisis Flag           Aggregate Clinical Level   
Metric                          No    Yes        Total       Clinical   
Year Screening Session                                                  
2020 0                         0.0    0.0        419.0          224.0   
     1                       908.0  119.0       1019.0          842.0   

Entity                                                                       \
Group                               Provider Role                             
Metric                 Sub-clinical  Care_manager Chaplain Clinician  Coach   
Year Screening Session                                                        
2020 0                        195.0           0.0     11.0     165.0  179.0   
     1                        177.0           2.0      7.0     629.0  162.0   

Entity                                                                 \
Group                                                                   
Metric                 Other Peer Psychiatrist Psychologist Undefined   
Year Screening Session                                                  
2020 0                   1.0  4.0         59.0          0.0       0.0   
     1                   0.0  6.0        213.0          0.0       0.0   

Entity                 Group Sessions Content Views                       \
Group                       Aggregate     Aggregate Content Type           
Metric                          Total         Total          App Article   
Year Screening Session                                                     
2020 0                            8.0        4359.0          0.0  1924.0   
     1                            3.0         694.0          0.0   365.0   

Entity                                            
Group                                             
Metric                 Podcast   Video Worksheet  
Year Screening Session                            
2020 0                   716.0  1523.0     196.0  
     1                   156.0   129.0      44.0

In [59]:
print(len(report_year_screen_pct_idx))
print(len(report_year_screen_pct_idx.columns))
report_year_screen_pct_idx.head(2)

12
66


Entity                  Accounts                              \
Group                  Aggregate                Account Type   
Metric                     Total Total_Year_Pct    Anonymous   
Year Screening Session                                         
2020 0                   12375.0         0.9138       9129.0   
     1                    1168.0         0.0862        356.0   

Entity                                                                \
Group                                                                  
Metric                 Anonymous_Year_Pct Verified Verified_Year_Pct   
Year Screening Session                                                 
2020 0                             0.9625   3246.0            0.7999   
     1                             0.0375    812.0            0.2001   

Entity                                                                   \
Group                  Account Source                                     
Metric                      Anonymous Anonymous_Year_Pct Email_password   
Year Screening Session                                                    
2020 0                         9129.0             0.9625            1.0   
     1                          356.0             0.0375            0.0   

Entity                                                       \
Group                                                         
Metric                 Email_password_Year_Pct Penn_key_sso   
Year Screening Session                                        
2020 0                                     1.0        430.0   
     1                                     0.0        112.0   

Entity                                                     \
Group                                                       
Metric                 Penn_key_sso_Year_Pct Penn_mpm_sso   
Year Screening Session                                      
2020 0                                0.7934          0.0   
     1                                0.2066          0.0   

Entity                                                                   \
Group                                                                     
Metric                 Penn_mpm_sso_Year_Pct Penn_sso Penn_sso_Year_Pct   
Year Screening Session                                                    
2020 0                                   NaN   2815.0            0.8009   
     1                                   NaN    700.0            0.1991   

Entity                 Screening Sessions                              \
Group                           Aggregate                Triage Level   
Metric                              Total Total_Year_Pct         Mild   
Year Screening Session                                                  
2020 0                                0.0            0.0          0.0   
     1                             1027.0            1.0        441.0   

Entity                                                                  \
Group                                                                    
Metric                 Mild_Year_Pct Moderate Moderate_Year_Pct Severe   
Year Screening Session                                                   
2020 0                           0.0      0.0               0.0    0.0   
     1                           1.0    488.0               1.0   98.0   

Entity                                                                 \
Group                                  Crisis Flag                      
Metric                 Severe_Year_Pct          No No_Year_Pct    Yes   
Year Screening Session                                                  
2020 0                             0.0         0.0         0.0    0.0   
     1                             1.0       908.0         1.0  119.0   

Entity                              Appointments                 \
Group                                  Aggregate                  
Metric                 Yes_Year_Pct        Total Total_Year_Pct   
Year Scree

In [60]:
print(len(report_year_screen_pct_col))
print(len(report_year_screen_pct_col.columns))
report_year_screen_pct_col.head(2)

12
66


Entity                  Accounts                               \
Group                  Aggregate                 Account Type   
Metric                     Total Total_Group_Pct    Anonymous   
Year Screening Session                                          
2020 0                   12375.0             1.0       9129.0   
     1                    1168.0             1.0        356.0   

Entity                                                                  \
Group                                                                    
Metric                 Anonymous_Group_Pct Verified Verified_Group_Pct   
Year Screening Session                                                   
2020 0                              0.7377   3246.0             0.2623   
     1                              0.3048    812.0             0.6952   

Entity                                                                    \
Group                  Account Source                                      
Metric                      Anonymous Anonymous_Group_Pct Email_password   
Year Screening Session                                                     
2020 0                         9129.0              0.7377            1.0   
     1                          356.0              0.3048            0.0   

Entity                                                        \
Group                                                          
Metric                 Email_password_Group_Pct Penn_key_sso   
Year Screening Session                                         
2020 0                               8.0808e-05        430.0   
     1                               0.0000e+00        112.0   

Entity                                                      \
Group                                                        
Metric                 Penn_key_sso_Group_Pct Penn_mpm_sso   
Year Screening Session                                       
2020 0                                 0.0347          0.0   
     1                                 0.0959          0.0   

Entity                                                                     \
Group                                                                       
Metric                 Penn_mpm_sso_Group_Pct Penn_sso Penn_sso_Group_Pct   
Year Screening Session                                                      
2020 0                                    0.0   2815.0             0.2275   
     1                                    0.0    700.0             0.5993   

Entity                 Screening Sessions                               \
Group                           Aggregate                 Triage Level   
Metric                              Total Total_Group_Pct         Mild   
Year Screening Session                                                   
2020 0                                0.0             NaN          0.0   
     1                             1027.0             1.0        441.0   

Entity                                                                    \
Group                                                                      
Metric                 Mild_Group_Pct Moderate Moderate_Group_Pct Severe   
Year Screening Session                                                     
2020 0                            NaN      0.0                NaN    0.0   
     1                         0.4294    488.0             0.4752   98.0   

Entity                                                                   \
Group                                   Crisis Flag                       
Metric                 Severe_Group_Pct          No No_Group_Pct    Yes   
Year Screening Session                                                    
2020 0                              NaN         0.0          NaN    0.0   
     1                           0.0954       908.0       0.8841  119.0   

Entity                               Appointments                  \
Group                                   Aggregate               

#### Screenings and Appointments

In [61]:
# Screening and appointments yearly report
screen_appt_idx_names = ['year','has_screen_sess','has_appt']
screen_appt_idx_labels = ['Year','Screening Session','Connect with Support']
screen_appt_idx_dict = dict(zip(screen_appt_idx_names,screen_appt_idx_labels))

# Freq
report_year_screen_appt = crosstab_master.groupby(axis=0,level=[screen_appt_idx_names]).sum()
report_year_screen_appt.index = report_year_screen_appt.index.rename(screen_appt_idx_dict)
report_year_screen_appt = report_year_screen_appt.sort_index(axis=0)

# Pct
report_year_screen_appt_pct_dict = get_rel_pct(report_year_screen_appt)
report_year_screen_appt_pct_idx = report_year_screen_appt_pct_dict['idx']
report_year_screen_appt_pct_col = report_year_screen_appt_pct_dict['col']

# Set df names
report_year_screen_appt.name = 'report_year_screen_appt'
report_year_screen_appt_pct_idx.name = 'report_year_screen_appt_pct_idx'
report_year_screen_appt_pct_col.name = 'report_year_screen_appt_pct_col'

# Save data
group_name = 'report_year_screen_appt'
group_label = 'Year - Screen and Appt'
group_data = [report_year_screen_appt,report_year_screen_appt_pct_idx,report_year_screen_appt_pct_col]
set_report_group_record(group_name,group_label,group_data)

In [62]:
print(len(report_year_screen_appt))
print(len(report_year_screen_appt.columns))
report_year_screen_appt.head(2)

24
33


Entity                                       Accounts                        \
Group                                       Aggregate Account Type            
Metric                                          Total    Anonymous Verified   
Year Screening Session Connect with Support                                   
2020 0                 0                      12013.0       8878.0   3135.0   
                       1                        362.0        251.0    111.0   

Entity                                                                     \
Group                                       Account Source                  
Metric                                           Anonymous Email_password   
Year Screening Session Connect with Support                                 
2020 0                 0                            8878.0            1.0   
                       1                             251.0            0.0   

Entity                                                                 \
Group                                                                   
Metric                                      Penn_key_sso Penn_mpm_sso   
Year Screening Session Connect with Support                             
2020 0                 0                           414.0          0.0   
                       1                            16.0          0.0   

Entity                                               Screening Sessions  \
Group                                                         Aggregate   
Metric                                      Penn_sso              Total   
Year Screening Session Connect with Support                               
2020 0                 0                      2720.0                0.0   
                       1                        95.0                0.0   

Entity                                                                    \
Group                                       Triage Level                   
Metric                                              Mild Moderate Severe   
Year Screening Session Connect with Support                                
2020 0                 0                             0.0      0.0    0.0   
                       1                             0.0      0.0    0.0   

Entity                                                       Appointments  \
Group                                       Crisis Flag         Aggregate   
Metric                                               No  Yes        Total   
Year Screening Session Connect with Support                                 
2020 0                 0                            0.0  0.0          0.0   
                       1                            0.0  0.0        419.0   

Entity                                                                   \
Group                                       Clinical Level                
Metric                                            Clinical Sub-clinical   
Year Screening Session Connect with Support                               
2020 0                 0                               0.0          0.0   
                       1                             224.0        195.0   

Entity                                                                        \
Group                                       Provider Role                      
Metric                                       Care_manager Chaplain Clinician   
Year Screening Session Connect with Support                                    
2020 0                 0                              0.0      0.0       0.0   
                       1                              0.0     11.0     165.0   

Entity                                                                      \
Group                                                                        
Metric                                       Coach Other Peer Psychiatrist   
Year Screening Session Connect with Support                                  
2020 0  

In [63]:
print(len(report_year_screen_appt_pct_idx))
print(len(report_year_screen_appt_pct_idx.columns))
report_year_screen_appt_pct_idx.head(2)

24
66


Entity                                       Accounts                      \
Group                                       Aggregate                       
Metric                                          Total Total_Screening_Pct   
Year Screening Session Connect with Support                                 
2020 0                 0                      12013.0              0.9707   
                       1                        362.0              0.0293   

Entity                                                    \
Group                                       Account Type   
Metric                                         Anonymous   
Year Screening Session Connect with Support                
2020 0                 0                          8878.0   
                       1                           251.0   

Entity                                                                        \
Group                                                                          
Metric                                      Anonymous_Screening_Pct Verified   
Year Screening Session Connect with Support                                    
2020 0                 0                                     0.9725   3135.0   
                       1                                     0.0275    111.0   

Entity                                                              \
Group                                                                
Metric                                      Verified_Screening_Pct   
Year Screening Session Connect with Support                          
2020 0                 0                                    0.9658   
                       1                                    0.0342   

Entity                                                      \
Group                                       Account Source   
Metric                                           Anonymous   
Year Screening Session Connect with Support                  
2020 0                 0                            8878.0   
                       1                             251.0   

Entity                                                               \
Group                                                                 
Metric                                      Anonymous_Screening_Pct   
Year Screening Session Connect with Support                           
2020 0                 0                                     0.9725   
                       1                                     0.0275   

Entity                                                      \
Group                                                        
Metric                                      Email_password   
Year Screening Session Connect with Support                  
2020 0                 0                               1.0   
                       1                               0.0   

Entity                                                                    \
Group                                                                      
Metric                                      Email_password_Screening_Pct   
Year Screening Session Connect with Support                                
2020 0                 0                                             1.0   
                       1                                             0.0   

Entity                                                    \
Group                                                      
Metric                                      Penn_key_sso   
Year Screening Session Connect with Support                
2020 0                 0                           414.0   
                       1                            16.0   

Entity                                                                  \
Group                                                                    
Metric                                      Penn_key_sso_Screening_Pct   
Year Screening Session Connect with Support                  

In [64]:
print(len(report_year_screen_appt_pct_col))
print(len(report_year_screen_appt_pct_col.columns))
report_year_screen_appt_pct_col.head(2)

24
66


Entity                                       Accounts                  \
Group                                       Aggregate                   
Metric                                          Total Total_Group_Pct   
Year Screening Session Connect with Support                             
2020 0                 0                      12013.0             1.0   
                       1                        362.0             1.0   

Entity                                                                        \
Group                                       Account Type                       
Metric                                         Anonymous Anonymous_Group_Pct   
Year Screening Session Connect with Support                                    
2020 0                 0                          8878.0              0.7390   
                       1                           251.0              0.6934   

Entity                                                                   \
Group                                                                     
Metric                                      Verified Verified_Group_Pct   
Year Screening Session Connect with Support                               
2020 0                 0                      3135.0             0.2610   
                       1                       111.0             0.3066   

Entity                                                      \
Group                                       Account Source   
Metric                                           Anonymous   
Year Screening Session Connect with Support                  
2020 0                 0                            8878.0   
                       1                             251.0   

Entity                                                           \
Group                                                             
Metric                                      Anonymous_Group_Pct   
Year Screening Session Connect with Support                       
2020 0                 0                                 0.7390   
                       1                                 0.6934   

Entity                                                      \
Group                                                        
Metric                                      Email_password   
Year Screening Session Connect with Support                  
2020 0                 0                               1.0   
                       1                               0.0   

Entity                                                                \
Group                                                                  
Metric                                      Email_password_Group_Pct   
Year Screening Session Connect with Support                            
2020 0                 0                                  8.3243e-05   
                       1                                  0.0000e+00   

Entity                                                    \
Group                                                      
Metric                                      Penn_key_sso   
Year Screening Session Connect with Support                
2020 0                 0                           414.0   
                       1                            16.0   

Entity                                                              \
Group                                                                
Metric                                      Penn_key_sso_Group_Pct   
Year Screening Session Connect with Support                          
2020 0                 0                                    0.0345   
                       1                                    0.0442   

Entity                                                    \
Group                                                      
Metric                                      Penn_mpm_sso   
Year Screening Session Connect with Support                
2020 0                 0 

# Save to .xlsx

In [65]:
#def get_cobalt_report(report_name):

row_pad = 6
report_name = 'cobalt_reports'
report_date = str(datetime.datetime.now().date()).replace('-','')
report_save_name = REPORT_PATH + report_name + '_' + report_date + '.xlsx'
with pd.ExcelWriter(report_save_name) as writer:
    workbook=writer.book
    for group_name in report_group_dict['group_order']:
        group_label = report_group_dict[group_name]['label']
        group_date = report_group_dict[group_name]['date']
        group_records = report_group_dict[group_name]['records']
        group_record_order = report_group_dict[group_name]['record_order']
        
        sheet_name = group_label
        worksheet=workbook.add_worksheet(sheet_name)
        writer.sheets[sheet_name] = worksheet
        
        start_row = 0
        for record in group_record_order:
            df = group_records[record]['data']
            num_rows = len(df)
            df.to_excel(writer,sheet_name=sheet_name,startrow=start_row,startcol=0)
            start_row += num_rows+row_pad

# Working Working Working

#### TO DO - 2023/02/26
* Report 1:
    * need all time **DONE**
    * need change index values (domain names)/name (Domain) **DONE**

* Report 2:
    * relative pcts **DONE**
    
#### General Questions moving forward
* Would pivot() or crosstab() be easier
* Would building an account dataset be easier?
* How to answer these questions: 
    * How many anonymous accts have an appt (supported - has_appt indicator)
    * How many appts have an anonymous acct (NOT supported - need anon indicator)

### Popular Content

In [63]:
popular_content.head(2)

,content_id,content_type_id,title,url,date_created,image_url,description,author,created,last_updated,owner_institution_id,deleted_flag,archived_flag,owner_institution_approval_status_id,other_institution_approval_status_id,visibility_id,duration_in_minutes,content_type_label_id,en_search_vector,view_count
0,09ab5264-3d8f-4e8e-be15-074e38528bf1,VIDEO,Tackling Burnout,https://youtu.be/okweTcWwe8c,2021-06-15 00:00:00+00:00,https://penncobalt.s3.us-east-2.amazonaws.com/...,"Dr. Laura Hamill, Chief Science Officer at Lim...",Dr. Laura Hamill,2021-06-15 20:08:10.643692+00:00,2022-10-19 15:19:42.026897+00:00,PENN,False,False,APPROVED,PENDING,PRIVATE,60.0,VIDEO,"'also':33 'behind':15 'burnout':2,16,63 'caus'...",792
1,0e722b38-b101-4a8c-8a2e-f34ff19137ea,PODCAST,Managing Compassion Fatigue,https://youtu.be/8JcVqJjZTbY,NaT,https://penncobalt.s3.us-east-2.amazonaws.com/...,"We have been dipping extensively into our ""wel...","Lily Brown, PhD",2022-01-24 15:05:57.739142+00:00,2023-02-13 15:33:08.651356+00:00,PENN,False,False,APPROVED,PENDING,PRIVATE,22.0,PODCAST,"'20':57 'anxieti':72 'ask':30 'brown':62,95,11...",770


In [24]:
popular_content.head(2)

,content_id,content_type_id,title,url,date_created,description,author,created,last_updated,owner_institution_id,deleted_flag,duration_in_minutes,en_search_vector,never_embed,shared_flag,search_terms,publish_start_date,publish_end_date,publish_recurring,published,file_upload_id,image_file_upload_id,view_count
0,09ab5264-3d8f-4e8e-be15-074e38528bf1,VIDEO,Tackling Burnout,https://youtu.be/okweTcWwe8c,2021-06-15 00:00:00+00:00,"Dr. Laura Hamill, Chief Science Officer at Lim...",Dr. Laura Hamill,2021-06-15 20:08:10.643692+00:00,2024-02-02 15:25:25.919577+00:00,PENN,False,60.0,"'also':33 'behind':15 'burnout':2,16,63 'caus'...",False,False,None,2021-06-15 00:00:00+00:00,None,False,True,None,90428f87-6406-4d14-b06f-a6a15c61c61c,792
1,0e722b38-b101-4a8c-8a2e-f34ff19137ea,PODCAST,Managing Compassion Fatigue,https://youtu.be/8JcVqJjZTbY,2022-01-24 15:05:57.739142+00:00,"We have been dipping extensively into our ""wel...","Lily Brown, PhD",2022-01-24 15:05:57.739142+00:00,2024-02-02 15:25:25.919577+00:00,PENN,False,22.0,"'20':57 'anxieti':72 'ask':30 'brown':62,95,11...",False,False,None,2022-01-24 15:05:57.739142+00:00,None,False,True,None,5ec98d65-dbcb-4579-ae1d-55a76dfd9159,770


In [ ]:
usecols = ['content_id','content_type_id','title','created','author','view_count']
popular_content.sort_values('view_count',ascending=False)[usecols].head(10)

In [ ]:
content.head(2)

In [ ]:
content_activity.head(2)

In [ ]:
popular_content.head(2)

In [ ]:
content_category = get_table_data(reporting_cursor, 'content_category')
content_category.head()

In [ ]:
category = get_table_data(reporting_cursor, 'category')
category.head()

In [ ]:
popular_content.merge(content_category, how='inner', on='content_id')

### Group Reservations

In [ ]:
group_session_reservation.head(2)

In [ ]:
reservations = group_session_reservation.groupby(['group_session_id']).count()[['group_session_reservation_id']].sort_values('group_session_reservation_id',ascending=False)
reservations = reservations.rename(columns={'group_session_reservation_id':'group_reservation_count'})

usecols = ['group_session_id','title','description','facilitator_name','start_date_time','end_date_time','seats','group_reservation_count']
reservations = group_session.merge(reservations, how='inner', left_on='group_session_id', right_on='group_session_id').sort_values('group_reservation_count',ascending=False)[usecols]

In [ ]:
print(len(reservations))
print(len(reservations.columns))
reservations.head(5)

In [ ]:
group_session

In [ ]:
activity_tracking

In [ ]:
activity_tracking[activity_tracking['activity_type_id']=='URL']

### Augmented reporting
* Focus areas: refining "active users", expanding set of "indicators" for filtering, account deduping (i.e. the age-old "problem" of # unique users vs # unique accounts in an app that allows "unlimited ananymous account creation...)

In [ ]:
exclude_idx = [item for item in report_year_domains.index if sum(item[1:])==0]
active_accounts = report_year_domains.loc[~report_year_domains.index.isin(exclude_idx)].copy()
active_accounts.groupby(axis=0, level=0).sum()

In [ ]:
report_year_screen.groupby(axis=0, level=[0,1]).sum()

In [ ]:
crosstab_master.groupby(axis=0,level=[0,2]).sum()

In [ ]:
crosstab_master.groupby(axis=0, level=[0,3]).sum()

In [ ]:
idx = pd.IndexSlice
crosstab_master.loc[idx['All Time',:,:,1,:,:],:].sort_index().groupby(axis=0, level=[0,3]).sum()

In [ ]:
id_list_anon = acct[(acct['account_source_id']=='ANONYMOUS')&(acct['has_appt']==1)].account_id.unique()
id_list_ver = acct[(acct['account_source_id']!='ANONYMOUS')&(acct['has_appt']==1)].account_id.unique()

print(len(id_list_anon))
print(len(appt[appt['account_id'].isin(id_list_anon)]))
print()
print(len(id_list_ver))
print(len(appt[appt['account_id'].isin(id_list_ver)]))

In [ ]:
# Convert all grouping cols to dummies for FULL set of indicators
pd.get_dummies(acct.account_source_id)

In [ ]:
# To start deduping accts?? 
print(len(acct['email_address'].unique()))
print()
print(len(acct[acct['account_source_id']!='ANONYMOUS']))
print(len(acct[(acct['account_source_id']!='ANONYMOUS') & (acct['email_address'].notnull())]))
print(len(acct[(acct['account_source_id']!='ANONYMOUS') & (acct['email_address'].notnull())].email_address.unique()))
print()
print(len(acct[acct['account_source_id']=='ANONYMOUS']))
print(len(acct[(acct['account_source_id']=='ANONYMOUS') & (acct['email_address'].notnull())]))
print(len(acct[(acct['account_source_id']=='ANONYMOUS') & (acct['email_address'].notnull())].email_address.unique()))

#### Monthly Avgs

In [ ]:
# To get monthly averages from yearly data (rather than using monthly data directly)
(crosstab[('Accounts','Aggregate','Total')]/crosstab.index.get_level_values(0).map(delta_months_dict)).groupby(axis=0,level=[0,2,3,4,5]).sum()

#### Selecting column subsets or "data entities"

In [ ]:
crosstab_domains.loc[:,(['Accounts'],['Account Source'])]

# Unused/Deprecated Code

In [ ]:
#acct['has_rec'] = acct['account_id'].apply(lambda x: 1 if x in acct_rec_ids else 0)
#acct['has_appt'] = acct['account_id'].apply(lambda x: 1 if x in acct_appt_ids else 0).sum()

In [ ]:
"""

# Data Meta
data_meta = {'acct':{'data':acct, 
                     'idx_groups':{}, 
                     'col_groups':['account_type']}, 
             'esc':{'data':esc, 
                    'idx_groups':{'has_esc':esc_acct_ids,'has_crisis':esc_crisis_acct_ids}, 
                    'col_groups':['escalation']}, 
             'appt':{'data':appt, 
                     'idx_groups':{'has_appt':appt_acct_ids}, 
                     'col_groups':['provider_type','support_role_id']}, 
             'group_res':{'data':group_res, 
                          'idx_groups':{'has_group_res':group_res_acct_ids}, 
                          'col_groups':[]}, 
             'content_views':{'data':content_views, 
                              'idx_groups':{'has_content_views':content_views_acct_ids}, 
                              'col_groups':[]}, 
}

"""

In [ ]:
"""

# Indicator columns for recommendations and appointments
acct['has_rec']=0
acct.loc[acct['account_id'].isin(acct_rec_ids),['has_rec']]=1
acct['has_appt']=0
acct.loc[acct['account_id'].isin(acct_appt_ids),['has_appt']]=1

acct_rec['has_rec']=0
acct_rec.loc[acct_rec['account_id'].isin(acct_rec_ids),['has_rec']]=1
acct_rec['has_appt']=0
acct_rec.loc[acct_rec['account_id'].isin(acct_appt_ids),['has_appt']]=1

appt['has_rec']=0
appt.loc[appt['account_id'].isin(acct_rec_ids),['has_rec']]=1
appt['has_appt']=0
appt.loc[appt['account_id'].isin(acct_appt_ids),['has_appt']]=1

"""

In [ ]:
"""

# Domains, categories, and metrics for column labels
account_metrics = ['Accounts','Accounts/Month','Pct Accounts SSO']
pageview_metrics = ['Pageviews','Pageviews/Month','Pageviews/Account']
appointment_metrics = ['Booked Appts','Booked Appts/Month','Booked Appts/Account',
                       'Pct Appts Complete','Pct Appts Clinical']
group_metrics = ['Groups','Groups/Month','Groups/Account',
                 'Group Reservations','Group Reservations/Month','Group Reservations/Account']
content_metrics = ['Content','Content/Month','Content/Account',
                   'Content Views','Content Views/Month','Content Views/Account']

category_meta = {'Accounts':account_metrics, 
                 'Pageviews':pageview_metrics, 
                 'Appointments':appointment_metrics, 
                 'Group Sessions':group_metrics, 
                 'Content Activity':content_metrics}

domain_meta = {'Site-Wide':['Accounts','Pageviews'], 
               'Connect with Support':['Accounts','Pageviews','Appointments'], 
               'In the Studio':['Accounts','Pageviews','Group Sessions'], 
               'On Your Time':['Accounts','Pageviews','Content Activity'], 
               'labels':['Site-Wide','Connect with Support','In the Studio','On Your Time']}

column_names = []
for dom in domain_meta['labels']:
    for cat in domain_meta[dom]:
        for met in category_meta[cat]:
            column_names.append((dom,cat,met))

"""

In [ ]:
"""
#### OLDOLDOLDOLD Generate Report Data OLDOLDOLDOLD ####

# Time groups
timeframe_names = ['2020 (04/09-12/31)','2021 (01/01-12/31)','2022 (01/01-08/24)','All Time']
timeframes = [[item] for item in sorted(list(account.year.unique()))]
timeframes.append(sorted(list(account.year.unique())))

start_dates = [cobalt_start_date, 
               pd.to_datetime('2021/01/01').tz_localize(tz=COBALT_TZ),
               pd.to_datetime('2022/01/01').tz_localize(tz=COBALT_TZ),
               cobalt_start_date]
end_dates = [pd.to_datetime('2020/12/31').tz_localize(tz=COBALT_TZ),
             pd.to_datetime('2021/12/31').tz_localize(tz=COBALT_TZ),
             current_date,
             current_date]
             
# Account ID groups
acct_ids = account['account_id'].unique()
appt_acct_ids = appointment['account_id'].unique()
group_acct_ids = group_session_reservation['account_id'].unique()
content_acct_ids = content_activity['account_id'].unique()

filter_names = ['Site-Wide','Connect with Support','In the Studio','On Your Time']
filter_id_lists = [acct_ids,appt_acct_ids,group_acct_ids,content_acct_ids]


def get_account_stats():
    meta = dict()
    results = dict()
    
    # Filter for timeframe
    for timeframe_name,timeframe,start_date,end_date in zip(timeframe_names,timeframes,start_dates,end_dates):
        days = (end_date - start_date).days
        seconds = (end_date - start_date).seconds
        delta_days = days + seconds/SECONDS_DAY
        
        temp_account_timeframe = account[account['year'].isin(timeframe)]
        
        # Filter for analytics domain and account type
        for filter_name,filter_id_list in zip(filter_names,filter_id_lists):
            temp_account_filter = temp_account_timeframe[temp_account_timeframe['account_id'].isin(filter_id_list)]
            temp_account_filter_ver = temp_account_filter[temp_account_filter['account_source_id']!='ANONYMOUS']
            temp_account_filter_anon = temp_account_filter[temp_account_filter['account_source_id']=='ANONYMOUS']

            num_accounts = len(temp_account_filter)
            num_accounts_month = round((num_accounts/delta_days)*30,2)
            num_accounts_day = round(num_accounts/delta_days,2)

            num_accounts_ver = len(temp_account_filter_ver)
            num_accounts_ver_month = round((num_accounts_ver/delta_days)*30,2)
            num_accounts_ver_day = round(num_accounts_ver/delta_days,2)

            num_accounts_anon = len(temp_account_filter_anon)
            num_accounts_anon_month = round((num_accounts_anon/delta_days)*30,2)
            num_accounts_anon_day = round(num_accounts_anon/delta_days,2)


            results[(timeframe_name,filter_name)] = {('Visitors','Total'):num_accounts,
                                                     ('Visitors','Avg/Month'):num_accounts_month,
                                                     ('Verified Visitors','Total'):num_accounts_ver,
                                                     ('Verified Visitors','Avg/Month'):num_accounts_ver_month,
                                                     ('Anon Visitors','Total'):num_accounts_anon,
                                                     ('Anon Visitors','Avg/Month'):num_accounts_anon_month}

            meta[(timeframe_name,filter_name)] = {'timeframe':timeframe,
                                                  'start_date':start_date,
                                                  'end_date':end_date,
                                                  'days':days,
                                                  'seconds':seconds,
                                                  'delta_days':delta_days}
        
    return results,meta


# Get critical cnalytics report
data,meta = get_account_stats()

data_df = pd.DataFrame(data).transpose()
meta_df = pd.DataFrame(meta).transpose()

reorder = [(filter_name,timeframe_name) for filter_name in filter_names for timeframe_name in timeframe_names]
data_swap_df = data_df.swaplevel(0,1).reindex(reorder)
meta_swap_df = meta_df.swaplevel(0,1).reindex(reorder)

# Save to csv
data_df.to_csv(CHART_PATH + 'critical_analytics_' + str(datetime.datetime.now().date()).replace('-','') + '.csv')
data_swap_df.to_csv(CHART_PATH + 'critical_analytics_swap_' + str(datetime.datetime.now().date()).replace('-','') + '.csv')


# Distribution of active visitors across analytics domains, per timeframe
print(len(data_df))
print(len(data_df.columns))
data_df

# comparison of analytics domains  over time
print(len(data_swap_df))
print(len(data_swap_df.columns))
data_swap_df

"""

In [ ]:
"""

#### OLDOLDOLDOLD Accounts OLDOLDOLDOLD #### 
# Group by indicator cols
acct_crosstab = acct.groupby(grouping+['account_type'])[['account_id']].count().unstack()

# Adjust indices and labeling
level_0_names = {'account_id':'accounts'}
level_1_names = {item:item.lower() for item in acct_crosstab.columns.get_level_values(1)}
acct_crosstab = acct_crosstab.rename(columns=level_0_names, level=0)
acct_crosstab = acct_crosstab.rename(columns=level_1_names, level=1)
acct_crosstab.columns = acct_crosstab.columns.rename(col_level_names)

# Add data for all time
temp_acct_crosstab = pd.concat({'All Time': acct_crosstab.groupby(axis=0,level=[1,2]).sum()}, names=['year'])
acct_crosstab = pd.concat([temp_acct_crosstab,acct_crosstab], axis=0)

acct_crosstab


#### OLDOLDOLDOLD Recommendations OLDOLDOLDOLD #### 
# Group by indicator cols
acct_rec_crosstab = acct_rec.groupby(grouping+['escalation'])[['instance_id']].count().unstack()
# Adjust indices and labeling
level_0_names = {'instance_id':'recommendations'}
level_1_names = {item:item.lower() for item in acct_rec_crosstab.columns.get_level_values(1)}
acct_rec_crosstab = acct_rec_crosstab.rename(columns=level_0_names, level=0)
acct_rec_crosstab = acct_rec_crosstab.rename(columns=level_1_names, level=1)
acct_rec_crosstab.columns = acct_rec_crosstab.columns.rename(col_level_names)
# Add data for all time
temp_acct_rec_crosstab = pd.concat({'All Time': acct_rec_crosstab.groupby(axis=0,level=[1,2]).sum()}, names=['year'])
acct_rec_crosstab = pd.concat([temp_acct_rec_crosstab,acct_rec_crosstab], axis=0)

acct_rec_crosstab


#### OLDOLDOLDOLD Appointments OLDOLDOLDOLD #### 
# Group by indicator cols
appt_clin_crosstab = appt.groupby(grouping+['provider_type'])[['appointment_id']].count().unstack()
appt_role_crosstab = appt.groupby(grouping+['support_role_id'])[['appointment_id']].count().unstack()
appt_crosstab = pd.concat([appt_clin_crosstab,appt_role_crosstab],axis=1).fillna(0)
# Adjust indices and labeling
level_0_names = {'appointment_id':'appointments'}
level_1_names = {item:item.lower() for item in appt_crosstab.columns.get_level_values(1)}
appt_crosstab = appt_crosstab.rename(columns=level_0_names, level=0)
appt_crosstab = appt_crosstab.rename(columns=level_1_names, level=1)
appt_crosstab.columns = appt_crosstab.columns.rename(col_level_names)
# Add data for all time
temp_appt_crosstab = pd.concat({'All Time': appt_crosstab.groupby(axis=0,level=[1,2]).sum()}, names=['year'])
appt_crosstab = pd.concat([temp_appt_crosstab,appt_crosstab], axis=0)

appt_crosstab


#### OLDOLDOLDOLD Combined OLDOLDOLDOLD #### 
crosstab = pd.concat([acct_crosstab,acct_rec_crosstab,appt_crosstab],axis=1).fillna(0)
crosstab

"""

In [ ]:
"""

report_year_idx_names = ['year','has_screen_sess','has_appt','has_group_res','has_content_views']
report_year_idx_labels = ['Year','Screening Session','Connect with Support','In the Studio','On Your Time']
report_year_idx_dixt = dict(zip(report_year_idx_names,report_year_idx_labels))

report_domain_idx_names = ['has_appt','has_group_res','has_content_views']
report_domain_idx_labels = ['Connect with Support','In the Studio','On Your Time']
report_domain_idx_dict = dict(zip(report_domain_idx_names,report_domain_idx_labels))

"""

In [ ]:
"""
# Split into individual domains for grouping and relabeling
domain1 = report_year_domains.loc[idx[:,1,:,:],:].groupby(axis=0, level=[0,1]).sum()
domain1 = domain1.rename(index={1:domain1.index.names[1]},level=1)
domain1.index = domain1.index.rename(['Year','Domain'])

domain2 = report_year_domains.loc[idx[:,:,1,:],:].groupby(axis=0, level=[0,2]).sum()
domain2 = domain2.rename(index={1:domain2.index.names[1]},level=1)
domain2.index = domain2.index.rename(['Year','Domain'])

domain3 = report_year_domains.loc[idx[:,:,:,1],:].groupby(axis=0, level=[0,3]).sum()
domain3 = domain3.rename(index={1:domain3.index.names[1]},level=1)
domain3.index = domain3.index.rename(['Year','Domain'])





site_wide = report_year_domains.groupby(axis=0, level=0).sum()
site_wide.index = pd.MultiIndex.from_product([site_wide.index, ['Site_wide']], names=['Year','Domain'])

cobalt_domains = pd.concat([site_wide,domain1,domain2,domain3]).sort_index()
cobalt_domains_swap = cobalt_domains.swaplevel(0,1).sort_index()


"""